### Perform Necessary Installations

In [ ]:
!pip install bs4
!pip install requests
!pip install pandas
!pip numpy

In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re
import numpy as np

### Define Helper Functions
We will use those throughout our notebook as they make our code resuable, extensible, and readable 

In [ ]:
def flatten_comprehension(matrix):
     return [item for row in matrix for item in row]


def extract_rows_from_tables(tables):
    rows = []
# Now, iterate over each table in the ResultSet
    for table in tables:
        # For each table, you can now call find_all on it
        rows.append(table.find_all('tr'))
        # Do something with rows, e.g., print them
        
    
    return rows

def extract_raw_data(column_data,recurse_thru_a = False, find_tags = 'td', extract_provider_links = False, pattern =''):
    raw_data = []
    for row in column_data:
        row_data = row.find_all(find_tags)
        individual_row_data = []
        
        if(recurse_thru_a == True):
            extracted_links = [data.find('a')['href'] if data.find('a') else data.text.strip() for data in row_data]
#             print("extracted links: ", extracted_links)
            link_to_page = extracted_links[0] if extracted_links else None
#             print(f'Link to page: {link_to_page}')
            individual_row_data.append(link_to_page)
        # print("row_data[0] = ", row_data[0])
        individual_row_data +=[data.text.strip() for data in row_data]
        
        if extract_provider_links == True:
            extracted_links = [data.find('a')['href'] if data.find('a') else data.text.strip() for data in row_data]
            for element in extracted_links:
                matches = re.findall(pattern, element)
                if(matches):
                    individual_row_data.append(element)

        raw_data.append(individual_row_data)
    return raw_data
def extract_providers_link(column_data, find_tags = 'td',  pattern = ''):
    raw_data = []
    for row in column_data:
        row_data = row.find_all(find_tags)
        individual_row_data = []
        
       
        extracted_links = [data.find('a')['href'] if data.find('a') else data.text.strip() for data in row_data]
        
        
#         print(extracted_links)
        for element in extracted_links:
            matches = re.findall(pattern, element)
            if(matches):
                # print(f"'{element}' contains {len(matches)} occurrence(s) of {pattern}")
                individual_row_data +=[data.text.strip() for data in row_data]
                individual_row_data.append(element)
                raw_data.append(individual_row_data)

                
#         individual_row_data +=[data.text.strip() for data in row_data]
#         print(f'{individual_row_data}')
    return raw_data
def append_data_to_df(df, raw_data):
    for data in raw_data:
#         print(f'data: {data}')
        length = len(df)
        df.loc[length] = data
    return df

def preprocess_extracted_sats(cont_sats_extracted, row_names, attr_index = 0):
    for index, sat in enumerate(cont_sats_extracted):
#         print("len(sat) = > ",len(sat))
#         print("len(row_names) = > ",len(sat))
        
        if len(sat) <= len(row_names) - 1:
#             print(f'found a channel "{sat}"')
            sat_dir = cont_sats_extracted[index - 1][attr_index]
#             print(f'New Attr "{sat_dir}"')
            sat.insert(attr_index, sat_dir)
            cont_sats_extracted[index] = sat
#             print(f'New channel "{cont_sats_extracted[index]}"')

    return cont_sats_extracted

def append_region_to_extracted_sats(cont_sats_extracted, continent):
    for index, sat in enumerate(cont_sats_extracted):
        cont_sats_extracted[index] =  sat + [continent]
    return cont_sats_extracted

In [ ]:
def extract_raw_data_alt(column_data,recurse_thru_a = False, find_tags = 'td', extract_provider_links = False, pattern =''):
    raw_data = []
    for row in column_data:
        row_data = row.find_all(find_tags)
        columns = row_data
        individual_row_data = []
        
        
        if(recurse_thru_a == True):
            extracted_links = [data.find('a')['href'] if data.find('a') else data.text.strip() for data in row_data]
            link_to_page = extracted_links[0] if extracted_links else None
            individual_row_data.append(link_to_page)

        individual_row_data +=[data.text.strip() for data in row_data]
        
       
        # print(len(columns))
        if len(columns) == 10:
            font = columns[0].find_all('font')[0]
            # print('font: ', font)
            
            br = font.find_all('br')
            # print('br: ', br)
            freq = None
            beam = None
            eirp = None
            for i, thing in enumerate(br):
                if i == 0:
                    freq = thing.previous_sibling.get_text(strip=True)
                elif i == 1:
                    beam = thing.previous_sibling.get_text(strip=True)
                else:
                    eirp = thing.next_sibling.get_text(strip=True) if thing.next_sibling else None
            # print(f'freq: {freq}, beam: {beam}, eirp: {eirp}')
        individual_row_data.append(freq)
        individual_row_data.append(beam)
        individual_row_data.append(eirp)
        
        if len(columns) == 10:
            fonts = columns[8].find_all('font')
            # print('fonts for network: ', fonts)
            fonts = [bs(str(data).replace('<br/>',',')).text for data in fonts]
            first_font = fonts[0] if fonts else None
            individual_row_data.append(first_font)
        else:
            fonts = columns[6].find_all('font')
            fonts = [bs(str(data).replace('<br/>',',')).text for data in fonts]
            first_font = fonts[0] if fonts else None
            individual_row_data.append(first_font)
            # individual_row_data += fonts.pop()
        # print('fonts: ', fonts)
                        
        # individual_row_data +=[data.text.strip() for data in row_data]
        
        # individual_row_data +=[BeautifulSoup(str(data).replace('<br/>',',')).text for data in row_data]
        
        if extract_provider_links == True:
            extracted_links = [data.find('a')['href'] if data.find('a') else data.text.strip() for data in row_data]
            for element in extracted_links:
                matches = re.findall(pattern, element)
                if(matches):
                    individual_row_data.append(element)

        raw_data.append(individual_row_data)
    return raw_data

## Extract Satellites for All Regions

### List Continents and concatenate with their url

In [ ]:
continents = ['asia', 'europe', 'atlantic', 'america']
baseurl = 'https://www.lyngsat.com/'

# Construct the URLs
endpoints = []
for c in continents:
    endpoints.append(f'{baseurl}{c}.html')
endpoints

### Scrape each Endpoint

In [ ]:
satellites = []
for index, c in enumerate(continents):
    page = requests.get(endpoints[index])
    soup = bs(page.text, 'html')
    
    
    satellites.append({f'{c}':soup})

In [ ]:
row_names = ['Position','Name','Frequency','Launch_Date', 'Region']
cont_sats_df = pd.DataFrame(columns = row_names)
# 
for index, cont in enumerate(continents):
#     print('continent: ',cont)
    cont_sats_raw = satellites[index][cont]
    table = cont_sats_raw.find_all('table')[11]
    
    column_data = table.find_all('tr')
    
    cont_sats_extracted = extract_raw_data(column_data)
    
    cont_sats_extracted = preprocess_extracted_sats(cont_sats_extracted, row_names)
    cont_sats_extracted = append_region_to_extracted_sats(cont_sats_extracted, cont)    

    cont_sats_df = append_data_to_df(cont_sats_df, cont_sats_extracted)
    
# Switch the first (position) with the second column (name)
cont_sats_df = cont_sats_df.iloc[:, [1, 0] + list(range(2, len(cont_sats_df.columns)))]


In [ ]:
cont_sats_df

### Get Launching Details

In [ ]:
launch_endpoint = baseurl + "asia.html"
tables = get_table(launch_endpoint)

launch_endpoint
# page = requests.get(launch_endpoint)
# soup = bs(page.text, 'html')
# launch_endpoint


In [ ]:
column_data = extract_rows_from_tables(tables)

In [ ]:
for i, table in enumerate(column_data):
#     print("table#",i)
    if(len(table) > 2):
        _temp = column_data[i]
        _temp = _temp[2:len(_temp)-1]
    #     print(_temp)
        column_data[i] = _temp

# Flattens the list. Now, we have a list of tr tags
column_data = flatten_comprehension(column_data)


In [ ]:
column_data

In [ ]:
sats_extracted = extract_raw_data(table)
sats_extracted

In [ ]:
sats_extracted = extract_raw_data(column_data)

sats_extracted = [[entry[2], entry[3]] for entry in sats_extracted]
row_names = ['Sat_Name', 'Rocket']
rockets_df = pd.DataFrame(columns = row_names)

rockets_df = append_data_to_df(rockets_df, sats_extracted)
# rockets_df

merged_df = pd.merge(cont_sats_df, rockets_df, left_on = 'Name', right_on = 'Sat_Name', how='left')
# merged_df = merged_df.fillna(None)

merged_df

In [ ]:
cont_sats_df[['Position_Longitude', 'Position_Direction']] = cont_sats_df['Position'].str.split('\u00b0', expand=True)
cont_sats_df.drop('Position', axis=1, inplace=True)

cont_sats_df


In [ ]:
cont_sats_df.to_csv('E:/AUC/23-24/Spring/Database/Project/CSV Files/Satellites.csv',index=False)

In [ ]:
continents = ['asia', 'europe', 'atlantic', 'america']
baseurl = 'https://www.lyngsat.com/tracker/'

# Construct the URLs
endpoints = []
for c in continents:
    endpoints.append(f'{baseurl}{c}.html')
endpoints

In [ ]:
sat_rockets = []
for index, c in enumerate(continents):
    page = requests.get(endpoints[index])
    soup = bs(page.text, 'html')    
    
    sat_rockets.append({f'{c}':soup})

In [ ]:
sat_rockets

In [ ]:
row_names = ['URL','Position','Name','Frequency', 'Launch Date', 'Region']
cont_sats_df = pd.DataFrame(columns=row_names)

# index = 0
# cont = 'asia'
for index, cont in enumerate(continents):

    cont_sats_rockets = sat_rockets[index][cont]
    # cont_sats_rockets = sat_rockets[2]['atlantic']

    # print(cont_sats_rockets)

    table = cont_sats_rockets.find_all('table')[11]
    #     print(table)

    column_data = table.find_all('tr')
    #     print(column_data)

    cont_sats_extracted = extract_raw_data(column_data, True)

    # cont_sats_extracted = preprocess_extracted_sats(cont_sats_extracted, row_names)
    cont_sats_extracted = append_region_to_extracted_sats(cont_sats_extracted, cont)    

    # print(len(cont_sats_extracted))


    cont_sats_df = append_data_to_df(cont_sats_df, cont_sats_extracted[1:])
    # cont_sats_df =  cont_sats_extracted
        
    
# Switch the first (position) with the second column (name)
cont_sats_df = cont_sats_df.iloc[:, [1, 0] + list(range(2, len(cont_sats_df.columns)))]
cont_sats_df

In [ ]:
cont_sats_df['URL']

In [ ]:
rocket_list = []
rocket_date = []
for index, cont in enumerate(cont_sats_df['URL']):
    url = cont_sats_df['URL'][index]
    page = requests.get(url)
    soup = bs(page.text, 'html')
#     soup.find_all('font')[13]

    entry  = soup.find_all('font')
    
    entry = ' '.join(map(str,entry))
    sat_name = cont_sats_df['Name'][index]

    x = re.search("launched with (.+) \d\d\d\d", entry)
    
    _date = re.search('\d{4}-\d{2}-\d{2}', entry)
    if _date:
        rocket_date.append(
            {
            f'{sat_name}': _date.group()
        })
        # return match.group()
    else:
        print(f'Couldnt find a launching DATE for {sat_name} with entry {entry}')
#     print(x.group(1))
    
    
    
    print(f'Satellite name: {sat_name}')
    if(x):
#         print(f'Found l')
        rocket_list.append(
            {
            f'{sat_name}': x.group(1)
        })
    else:
        print(f'Couldnt find a launching rocket for {sat_name} with entry {entry}')
        
    print(f'Processed {index + 1} Satellites out of {len(cont_sats_df.index)}')
    

In [ ]:
for thing in rocket_list:
    print(thing)


In [ ]:
rocket_date

In [ ]:
rocket_list


list_of_lists = [list(d.items())[0] for d in rocket_list]
# new_list
row_names = ['Sat_name', 'L_Rocket']

list_of_lists_date = [list(d.items())[0] for d in rocket_date]

rockets_date_df = pd.DataFrame(list_of_lists_date, columns=['Sat_name', 'L_Date'])

rockets_df = pd.DataFrame(list_of_lists,columns = row_names)

rockets_df = pd.merge(left=rockets_df, right=rockets_date_df, left_on='Sat_name', right_on='Sat_name')
# rockets_df['Date'].isna

# new_rockets_df


In [ ]:
rockets_df['L_Date']

In [ ]:
# rockets_df['L_Rocket']

rockets_df.to_csv('C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/v6_Satellites_Rockets.csv', index = False)

In [ ]:
sat_df = pd.read_csv('E:/AUC/23-24/Spring/Database/Project/CSV Files/Satellites.csv')
# rockets_df = pd.read_csv('E:/AUC/23-24/Spring/Database/Project/CSV Files/Satellites_Rockets.csv')

# merged_df = pd.merge(sat_df, rockets_df, left_on = 'Name', right_on = 'Sat_name', how = 'left')
# merged_df = merged_df.drop(columns=['Sat_name'])
rockets_df = pd.read_csv('C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/v6_Satellites_Rockets.csv')

rockets_df = rockets_df[['Sat_name','L_Date']]
merged_df = pd.read_csv('E:/AUC/23-24/Spring/Database/Project/CSV Files/Satellites_Rockets.csv')

final_df = pd.merge(left=merged_df, right=rockets_df, left_on='Name', right_on='Sat_name', how='left')

final_df.shape
# merged_df.shape

In [ ]:
# final_df['L_Date'].isna().value_counts()
final_df.drop(columns=['Sat_name', 'Frequency','Launch_Date'], inplace=True)

In [ ]:
# display_max_length(final_df)
final_df['L_Date'].fillna('1900-01-01',inplace=True)

In [ ]:
final_df.iloc[22]

In [ ]:
# final_df[final_df['Name'].str.contains('T9S')]
final_df.to_csv('C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/v6_final_satellites.csv', index=False)

## Scrape Channels and Providers
For each satellite, we:
1. Scrape the Channels/ Providers records, treating them as one.
    a. Identify the Providers and merge with the original dataframe
    b. Save that
2. Assign Providers to Channels
3. Clean the Dataframes
    a. Split the System/SR/FEC columns
    b. Extract the languages
    c. Extract the Frequency

We begin by defining *helper functions* & *attributes*

In [ ]:
# Retrieves data tables from a given url
def get_table(url):
    
    page = requests.get(url)
    soup = bs(page.text, 'html')

    table = soup.find_all('table',{'border':"", 'cellpadding':"0", 'cellspacing':"0", 'width':"720"} )

    return table


In [ ]:
baseurl = 'https://www.lyngsat.com/'
# Contains the names of our Satellites
merged_df = pd.read_csv('E:/AUC/23-24/Spring/Database/Project/CSV Files/Satellites_Rockets.csv')

In [ ]:

# Prepare satellite names for incorporation into URLs

raw_sat_names = [link.replace(" ", "-") for link in merged_df['Name']]
raw_sat_names = [link.replace("'", "" ) for link in raw_sat_names]
raw_sat_names = [link.replace("ü", "u" ) for link in raw_sat_names]
raw_sat_names = [link.replace("/", "-" ) for link in raw_sat_names]
raw_sat_names = [link.replace("Ä", "A" ) for link in raw_sat_names]

sat_url_names = [re.split('-\(',link)[0] for link in raw_sat_names]
# baseurl

In [ ]:
sat_url_names

In [ ]:
very_raw_sat_urls = merged_df['Name']

sat_names_dict = dict(zip(sat_url_names, very_raw_sat_urls))
sat_names_dict

def get_normal_sat_name(safe_sat_name):
    return sat_names_dict.get(safe_sat_name)

### Scrape Channels & Providers

In [ ]:
## ALT SINGLE CHANNEL

## SCRAPE CHANNELS + PROVIDERS

# for index, sat_name in enumerate(sat_url_names):

index = 1
sat_name = 'Intelsat-18'
url = f'{baseurl}{sat_name}.html'
print(f'{index + 1} Processing satellite {sat_name} with url {url}')

# Scrape endpoint, retrieving all data tables 
sat_table = get_table(url)

# Retrieve all tr's from the returned tables
# Returns a list of lists (/table)
column_data = extract_rows_from_tables(sat_table)

for i, table in enumerate(column_data):
#     print("table#",i)
    if(len(table) > 2):
        _temp = column_data[i]
        _temp = _temp[2:len(_temp)-1]
    #     print(_temp)
        column_data[i] = _temp

# Flattens the list. Now, we have a list of tr tags
column_data = flatten_comprehension(column_data)

# Clean and extract the data values from the tags
channels_extracted = extract_raw_data_alt(column_data)
chans_links = extract_providers_link(column_data,pattern='.*tvchannels|.*radiochannels')
chan_links_df = pd.DataFrame(data=chans_links)
chan_links_df = chan_links_df.iloc[:, [1,8]]
chan_links_df.rename(columns ={1: "Channel", 8: "Channel_URL"}, inplace=True)
# print(chan_links_df)
networks_extracted = extract_providers_link(column_data, pattern='.*providers')



# Define the columns for our Main Dataframe
row_names = ['Freq/beam','SR/FEC', 'SID', 'Provider/Channel','undef','Compression','VPID','Audio', 'Encryption', 'Src_Updated', 'freq_alt', 'beam_alt', 'eirp_alt','encryption_alt']

# Define the columns for our Networks Dataframe
netw_row_names = ['Freq/beam','SR/FEC', 'SID', 'Provider/Channel','undef','Compression','VPID','Audio', 'Encryption', 'Src_Updated', 'Provider_URL']

# Construct the Network Dataframe
netw_df = pd.DataFrame(networks_extracted, columns = netw_row_names)

# Add Frequency/Beam to our channels (as inherited from the preceeding element)
channels_extracted = preprocess_extracted_sats(channels_extracted, row_names)

# Add System/SR/FEC to our channels (as inherited from the preceeding element)
channels_extracted = preprocess_extracted_sats(channels_extracted, row_names, 1)

# Construct the Main Dataframe
sat_df = pd.DataFrame(channels_extracted, columns = row_names)    

# Refine the Networks Dataframe to remove clutter/ redundant attributes
netw_df = netw_df[['Provider/Channel', 'Provider_URL']]

# Merge the Main and Network Dataframes
merged_df = pd.merge(left=sat_df, right=netw_df, how='outer',left_on='Provider/Channel', right_on='Provider/Channel')

# Adjust the filename to be safe (for saving the file on Windows)
safe_sat_name = sat_name.replace('/', '-') # Replace '/' with '_'
merged_df = pd.merge(left=merged_df, right=chan_links_df, left_on='Provider/Channel', right_on='Channel',how='outer')
# Save the merged Dataframe
merged_df.to_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v4/{index + 1}_{safe_sat_name}_channels.csv',index=False)


# temp_df = 
# print(temp_df)


In [ ]:
## ALT

## SCRAPE CHANNELS + PROVIDERS

for index, sat_name in enumerate(sat_url_names):

    url = f'{baseurl}{sat_name}.html'
    print(f'{index + 1} Processing satellite {sat_name} with url {url}')

    # Scrape endpoint, retrieving all data tables 
    sat_table = get_table(url)

    # Retrieve all tr's from the returned tables
    # Returns a list of lists (/table)
    column_data = extract_rows_from_tables(sat_table)

    for i, table in enumerate(column_data):
    #     print("table#",i)
        if(len(table) > 2):
            _temp = column_data[i]
            _temp = _temp[2:len(_temp)-1]
        #     print(_temp)
            column_data[i] = _temp

    # Flattens the list. Now, we have a list of tr tags
    column_data = flatten_comprehension(column_data)
   
    # Clean and extract the data values from the tags
    channels_extracted = extract_raw_data_alt(column_data)
    
    
    
    networks_extracted = extract_providers_link(column_data, pattern='.*providers')
    


    # Define the columns for our Main Dataframe
    row_names = ['Freq/beam','SR/FEC', 'SID', 'Provider/Channel','undef','Compression','VPID','Audio', 'Encryption', 'Src_Updated', 'freq_alt', 'beam_alt', 'eirp_alt','encryption_alt']
    
    # Define the columns for our Networks Dataframe
    netw_row_names = ['Freq/beam','SR/FEC', 'SID', 'Provider/Channel','undef','Compression','VPID','Audio', 'Encryption', 'Src_Updated', 'Provider_URL']
    
    # Construct the Network Dataframe
    netw_df = pd.DataFrame(networks_extracted, columns = netw_row_names)
    
    # Add Frequency/Beam to our channels (as inherited from the preceeding element)
    channels_extracted = preprocess_extracted_sats(channels_extracted, row_names)

    # Add System/SR/FEC to our channels (as inherited from the preceeding element)
    channels_extracted = preprocess_extracted_sats(channels_extracted, row_names, 1)

    # Construct the Main Dataframe
    sat_df = pd.DataFrame(channels_extracted, columns = row_names)    
    
    # Refine the Networks Dataframe to remove clutter/ redundant attributes
    netw_df = netw_df[['Provider/Channel', 'Provider_URL']]

    # Merge the Main and Network Dataframes
    merged_df = pd.merge(left=sat_df, right=netw_df, how='outer',left_on='Provider/Channel', right_on='Provider/Channel')

    # Adjust the filename to be safe (for saving the file on Windows)
    safe_sat_name = sat_name.replace('/', '-') # Replace '/' with '_'

    try:
        chans_links = extract_providers_link(column_data,pattern='.*tvchannels|.*radiochannels')
        chan_links_df = pd.DataFrame(data=chans_links)
        chan_links_df = chan_links_df.iloc[:, [1,8]]
        chan_links_df.rename(columns ={1: "Channel", 8: "Channel_URL"}, inplace=True)
        merged_df = pd.merge(left=merged_df, right=chan_links_df, left_on='Provider/Channel', right_on='Channel',how='outer')
    except:
        print(f'Problems with satellite: {safe_sat_name}')


    # Save the merged Dataframe
    merged_df.to_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v5/{index + 1}_{safe_sat_name}_channels.csv',index=False)



In [ ]:
## SCRAPE CHANNELS + PROVIDERS

for index, sat_name in enumerate(sat_url_names):

    url = f'{baseurl}{sat_name}.html'
    print(f'{index + 1} Processing satellite {sat_name} with url {url}')

    # Scrape endpoint, retrieving all data tables 
    sat_table = get_table(url)

    # Retrieve all tr's from the returned tables
    # Returns a list of lists (/table)
    column_data = extract_rows_from_tables(sat_table)

    for i, table in enumerate(column_data):
    #     print("table#",i)
        if(len(table) > 2):
            _temp = column_data[i]
            _temp = _temp[2:len(_temp)-1]
        #     print(_temp)
            column_data[i] = _temp

    # Flattens the list. Now, we have a list of tr tags
    column_data = flatten_comprehension(column_data)
   
    # Clean and extract the data values from the tags
    channels_extracted = extract_raw_data(column_data)
    networks_extracted = extract_providers_link(column_data, pattern='.*providers')
    


    # Define the columns for our Main Dataframe
    row_names = ['Freq/beam','SR/FEC', 'SID', 'Provider/Channel','undef','Compression','VPID','Audio', 'Encryption', 'Src_Updated']
    
    # Define the columns for our Networks Dataframe
    netw_row_names = ['Freq/beam','SR/FEC', 'SID', 'Provider/Channel','undef','Compression','VPID','Audio', 'Encryption', 'Src_Updated', 'Provider_URL']
    
    # Construct the Network Dataframe
    netw_df = pd.DataFrame(networks_extracted, columns = netw_row_names)
    
    # Add Frequency/Beam to our channels (as inherited from the preceeding element)
    channels_extracted = preprocess_extracted_sats(channels_extracted, row_names)

    # Add System/SR/FEC to our channels (as inherited from the preceeding element)
    channels_extracted = preprocess_extracted_sats(channels_extracted, row_names, 1)

    # Construct the Main Dataframe
    sat_df = pd.DataFrame(channels_extracted, columns = row_names)    
    
    # Refine the Networks Dataframe to remove clutter/ redundant attributes
    netw_df = netw_df[['Provider/Channel', 'Provider_URL']]

    # Merge the Main and Network Dataframes
    merged_df = pd.merge(left=sat_df, right=netw_df, how='outer',left_on='Provider/Channel', right_on='Provider/Channel')

    # Adjust the filename to be safe (for saving the file on Windows)
    safe_sat_name = sat_name.replace('/', '-') # Replace '/' with '_'

    # Save the merged Dataframe
    merged_df.to_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v2/{index + 1}_{safe_sat_name}_channels.csv',index=False)



### Assign Providers to Channels

In [ ]:
## ALT

## SCRAPE CHANNELS + PROVIDERS
# index = 0
# sat_name = 'NSS-9'
for index, sat_name in enumerate(sat_url_names):
    
    url = f'{baseurl}{sat_name}.html'
    print(f'{index + 1} Processing satellite {sat_name} with url {url}')

    # Scrape endpoint, retrieving all data tables 
    sat_table = get_table(url)

    # Retrieve all tr's from the returned tables
    # Returns a list of lists (/table)
    column_data = extract_rows_from_tables(sat_table)

    for i, table in enumerate(column_data):
    #     print("table#",i)
        if(len(table) > 2):
            _temp = column_data[i]
            _temp = _temp[2:len(_temp)-1]
        #     print(_temp)
            column_data[i] = _temp

    # Flattens the list. Now, we have a list of tr tags
    column_data = flatten_comprehension(column_data)

    # Clean and extract the data values from the tags
    channels_extracted = extract_raw_data_alt(column_data)



    networks_extracted = extract_providers_link(column_data, pattern='.*providers')



    # Define the columns for our Main Dataframe
    row_names = ['Freq/beam','SR/FEC', 'SID', 'Provider/Channel','undef','Compression','VPID','Audio', 'Encryption', 'Src_Updated', 'system_alt', 'sr_alt', 'fec_alt', 'freq_alt', 'beam_alt', 'eirp_alt','encryption_alt']

    # Define the columns for our Networks Dataframe
    netw_row_names = ['Freq/beam','SR/FEC', 'SID', 'Provider/Channel','undef','Compression','VPID','Audio', 'Encryption', 'Src_Updated', 'Provider_URL']

    # Construct the Network Dataframe
    netw_df = pd.DataFrame(networks_extracted, columns = netw_row_names)

    # Add Frequency/Beam to our channels (as inherited from the preceeding element)
    channels_extracted = preprocess_extracted_sats(channels_extracted, row_names)

    # Add System/SR/FEC to our channels (as inherited from the preceeding element)
    channels_extracted = preprocess_extracted_sats(channels_extracted, row_names, 1)

    # Construct the Main Dataframe
    sat_df = pd.DataFrame(channels_extracted, columns = row_names)
 
    sat_df[['system_alt', 'sr_alt', 'fec_alt']] = sat_df[['system_alt', 'sr_alt', 'fec_alt']].fillna(method='ffill')    
    # merged_sat_df['Frequency'] = merged_sat_df['Frequency'].fillna(method='ffill')


    # Refine the Networks Dataframe to remove clutter/ redundant attributes
    netw_df = netw_df[['Provider/Channel', 'Provider_URL']]

    # Merge the Main and Network Dataframes
    merged_df = pd.merge(left=sat_df, right=netw_df, how='outer',left_on='Provider/Channel', right_on='Provider/Channel')

    # Adjust the filename to be safe (for saving the file on Windows)
    safe_sat_name = sat_name.replace('/', '-') # Replace '/' with '_'

    try:
        chans_links = extract_providers_link(column_data,pattern='.*tvchannels|.*radiochannels')
        # print(chans_links)
        chan_links_df = pd.DataFrame(data=chans_links)
        chan_links_df = chan_links_df.iloc[:, [1,8]]
        chan_links_df.rename(columns ={1: "Channel", 8: "Channel_URL"}, inplace=True)
        merged_df = pd.merge(left=merged_df, right=chan_links_df, left_on='Provider/Channel', right_on='Channel',how='outer')
    except:
        print(f'Problems with satellite: {safe_sat_name}')
    
    try:
        pkgs_extracted = extract_providers_link(column_data, pattern='.*packages')
        if(len(pkgs_extracted) > 0):
            pkgs_extracted_df = pd.DataFrame(data=pkgs_extracted)
            pkgs_extracted_df= pkgs_extracted_df.iloc[:, [3,21]]
            pkgs_extracted_df.rename(columns ={3: "Pkg_Name", 21: "Pkg_URL"}, inplace=True)
            merged_df = pd.merge(left=merged_df, right=pkgs_extracted_df, left_on='Provider/Channel', right_on='Pkg_Name',how='outer')

        # print(chan_links_df.shape)
        # merged_df.pd_merge(left=merged_df, right )

    except: 
        print(f'problem with pkgs extraction')
    # Save the merged Dataframe
    merged_df.to_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v6/{index + 1}_{safe_sat_name}_channels.csv',index=False)



In [ ]:

for index, sat_name in enumerate(sat_url_names):
#     index = 1
#     sat_name = 'Intelsat-18'
    safe_sat_name = sat_name.replace('/', '-') # Replace '/' with '_'
    my_df = pd.read_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v6/{index + 1}_{safe_sat_name}_channels.csv')

    # if len(my_df.index) == 0:
    #     continue
    print(f'{index + 1} Processing sat: {safe_sat_name}')

    my_df['is_Provider'] = np.where(my_df['Provider_URL'].astype(str).str.contains('http', regex=True, na=False), True, False)

    providers_df = my_df[my_df['is_Provider'] == True]
    channels_df = my_df[my_df['is_Provider'] == False]

    my_df['Provider'] = None


    for i in range(len(my_df.index)):
        isProvider = my_df.loc[i, 'is_Provider']
    
        if isProvider == True:
            my_df.loc[i, 'Provider'] = my_df.loc[i, 'Provider/Channel']
            continue

        if i > 0:
            my_df.loc[i, 'Provider'] = my_df.loc[i-1, 'Provider'] 
    my_df.to_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v6/assigned/{index + 1}_{safe_sat_name}_channels.csv',index=False)

# my_df

### Clean our Dataframes

##### Define Helper Functions

In [ ]:


# Function to extract string until the last capital character
def extract_until_last_capital(s):
    match = re.search(r'(.*[A-Z])', s)
    if match:
        return match.group(1)
    return s
def find_pattern_and_join(entry):
    matches = re.findall(r"([A-Z][a-z]+)", entry)
    return ' '.join(matches) # Join the matches into a single string
def split_beam_eirp(entry, beam_eirp_index = 0):
    temp = re.split(r'(?<=\d)(?=\D)', entry)
    if beam_eirp_index == 1:
        # Check if the value is not None
        if(len(temp) > 1):
            return temp[beam_eirp_index] 
        else:
            'None'
    else:
        return temp[beam_eirp_index]

In [ ]:
skipped_sats_url = []

#### Perform The Cleaning

In [ ]:


for index, sat_name in enumerate(sat_url_names):
    safe_sat_name = sat_name.replace('/', '-') # Replace '/' with '_'
    hor_df = pd.read_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v6/assigned/{index + 1}_{safe_sat_name}_channels.csv')

    print(f'{index + 1} Processing sat: {safe_sat_name}')

    if len(hor_df.index) == 0:
        url = f'{baseurl}{sat_name}.html'
        skipped_sats_url.append(url)
        hor_df.to_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v6/assigned/cleaned/{index+1}_{safe_sat_name}_channels.csv', index=False)
        continue
    
    # Drop irrelevant columns
    hor_df.drop(columns=['undef','Src_Updated', 'Freq/beam', 'Encryption'], inplace = True)  
    
    # try:
    #     hor_df['FEC'] = hor_df['SR/FEC'].str[-3:]
    #     hor_df['SR'] = hor_df['SR/FEC'].str[-8:-3].str.extract('(\d+)')
    #     hor_df['SYSTEM'] = hor_df['SR/FEC'].str[0:6]
    # except:
    #     print("PROBLEM YO")

    # Drop the now-old composite column
    hor_df.drop('SR/FEC', axis=1, inplace=True)

    # Extract Languages
    hor_df['Audio'] = hor_df['Audio'].astype(str)

    hor_df['Languages'] = hor_df['Audio'].apply(find_pattern_and_join)
    
    hor_df.drop('Audio', axis=1, inplace=True)
    
    # if 'A' in df.columns:
    if 'Pkg_URL' in hor_df.columns:
        hor_df.drop(hor_df[hor_df['Pkg_URL'].notna()].index, inplace=True)
    # Split the Composite Frequency column    
#     try:
# #         extract_until_last_capital
#         hor_df['Freq'] = hor_df['Freq/beam'].str[0:7].apply(extract_until_last_capital)
#     except:
#         print("PROBLEMSSSSSS")
    
    hor_df.to_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v6/assigned/cleaned/{index+1}_{safe_sat_name}_channels.csv', index=False)    

In [ ]:
# View the URLs that were skipped from cleaning due to lack of data
skipped_sats_url

In [ ]:
# View the current output for a cleaned dataframe
index = 1
safe_sat_name = 'Intelsat-18'

my_df = pd.read_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v5/assigned/cleaned/{index+1}_{safe_sat_name}_channels.csv')


In [ ]:
my_df.head()

## Get Country for Provider

In [ ]:


# Retrieves data tables from a given url
def get_country(url):
    
    page = requests.get(url)
    soup = bs(page.text, 'html')

    table = soup.find_all('table',{'width':"700"} )

    return table
def extract_country(url):
    my_pattern = "/\D\D/"
    url = str(url)
    # print('url => ', url)
    matches = re.search(my_pattern, url)
    if matches:
        country = matches.group() # Assuming the country is the first captured group
        country = country[1:-1]
        return country
    else:
        return None # or any default value you prefer



In [ ]:
new_df = pd.DataFrame(columns = ['Sattelite', 'Provider', 'Country'])


In [ ]:

## Extract Channel Country
x_df = None

netw_flag = True
for index, sat_name in enumerate(sat_url_names):
    
    safe_sat_name = sat_name.replace('/', '-') # Replace '/' with '_'
    new_col_name = 'Provider_Country'
    ref_col = 'Provider_URL'
    dir_name = 'prov_country'
    if netw_flag == False:
        new_col_name = 'Channel_Country'
        ref_col = 'Channel_URL'
        dir_name = 'chan_country'
        
        # May cause problems; remember to change the chan_country dir
    hor_df = pd.read_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v6/assigned/cleaned/chan_country/{index + 1}_{safe_sat_name}_channels.csv')
    
    # hor_df['Channel_Country'] = None
    print(f'{index + 1} Processing sat: {safe_sat_name}')

    if len(hor_df.index) == 0:
        # url = f'{baseurl}{sat_name}.html'
        # skipped_sats_url.append(url)
        print(f'Skipping sattelite: {safe_sat_name}')
        hor_df.to_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v6/assigned/cleaned/{dir_name}/{index+1}_{safe_sat_name}_channels.csv', index=False)
        continue
    # prov_urls = hor_df['Provider_URL'].dropna().unique()
    

    
    # my_pattern = "/\D\D/"
    # for url in prov_urls:
        
    #     print(f'Processing Provider: {url}')
    #     provider = hor_df[hor_df['Provider_URL'] == url].iloc[0]['Provider/Channel']

    #     matches = re.search(my_pattern, url)
    #     matches = matches.group()
    #     country = matches[1:-1]
    #     new_df.loc[len(new_df)] = [safe_sat_name, provider ,country]
    
    # for url in prov_urls
    
    
    try:
        hor_df[new_col_name] = hor_df[ref_col].apply(extract_country)
        hor_df.to_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v6/assigned/cleaned/{dir_name}/{index +1}_{safe_sat_name}_channels.csv', index=False)
    except:
        print('problem yo')
        hor_df.to_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v6/assigned/cleaned/{dir_name}/{index +1}_{safe_sat_name}_channels.csv', index=False)
    # print(x_df.head())
        # table = get_country(url)
        # column_data = extract_rows_from_tables(table)
        # # Flattens the list. Now, we have a list of tr tags
        # column_data = flatten_comprehension(column_data)
        # col = extract_raw_data(column_data)
        # if(len(col) > 1):
        #     country = col[0][0].split('\n')[-1]
        #     new_df.loc[len(new_df)] = [safe_sat_name, provider ,country]


In [ ]:
x_df
# new_df.to_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v4/assigned/cleaned/prov_country/prov_with_country_from_url.csv', index=False)

In [ ]:

new_df = pd.DataFrame(columns=['Satellite', 'Provider', 'Country'])

prov_urls = my_df['Provider_URL'].dropna().unique()

# prov_urls = ['https://www.lyngsat.com/tvchannels/us/AFN-Prime-Atlantic.html']
for url in prov_urls:
    table = get_country(url)
    column_data = extract_rows_from_tables(table)

    # Flattens the list. Now, we have a list of tr tags
    column_data = flatten_comprehension(column_data)
    col = extract_raw_data(column_data)
    if(len(col) > 1):
        provider = my_df[my_df['Provider_URL'] == url].iloc[0]['Provider/Channel']
        country = col[0][0].split('\n')[-1]
        new_df.loc[len(new_df)] = [safe_sat_name, provider ,country]

In [ ]:
new_df = pd.read_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v6/assigned/cleaned/prov_country/1_NSS-9_channels.csv' )

# new_df['Sat_Name'] = 'ali'


In [ ]:
all_channels = pd.DataFrame(columns=new_df.columns)
dfs_list = []

In [ ]:
# all_channels
for index, sat_name in enumerate(sat_url_names):
    print(sat_name)
    safe_sat_name = sat_name.replace('/', '-') # Replace '/' with '_'
    curr_df = pd.read_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v6/assigned/cleaned/prov_country/{index + 1}_{sat_name}_channels.csv')
    curr_df['Sattelite_Name'] = safe_sat_name
    # all_channels = pd.concat([all_channels, curr_df])
    dfs_list.append(curr_df)


In [ ]:
all_channels = pd.concat(dfs_list, ignore_index=True)
all_channels.shape

In [ ]:
all_channels.drop_duplicates(inplace=True)

In [ ]:
all_channels.shape

In [ ]:
# all_channels.drop_duplicates().head()
# all_channels.drop(columns=['Src_Updated', 'Encryption', 'Audio', 'undef','SR/FEC', 'Freq/beam'], inplace=True)

all_channels.to_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/all_channels_raw_v6.csv',index=False)

In [ ]:
all_channels = pd.read_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/all_channels_raw_v6.csv')

all_channels.shape

In [ ]:

feeds = all_channels[all_channels['Provider/Channel'].astype(str).str.contains('^\(f.*\)|^\[')]

indices = feeds.index.values.tolist()  # an ndarray method, you probably shouldn't depend on this


In [ ]:
# feeds
all_channels.drop(axis = 0, index = indices, inplace=True)

In [ ]:
# print(temp_all.shape)
print(all_channels.shape)

In [ ]:
indices = all_channels[all_channels['Provider/Channel'].astype(str).str.contains('^@')].index.values.tolist()

In [ ]:
all_channels.drop(axis=0, index=indices, inplace=True)

In [ ]:
all_channels.head(3)
all_channels.shape

# all_channels[all_channels['is_Provider'] == False]

# all_channels.head(12)

In [ ]:
# channel_instances_df = all_channels[~all_channels['Channel_URL'].isna()]
channel_instances_df = all_channels[all_channels['is_Provider'] == False]
channel_instances_df = channel_instances_df[channel_instances_df['Provider/Channel'].notna()]

channel_instances_df.shape

In [ ]:

unique_channels = channel_instances_df.drop_duplicates(subset=['Provider/Channel'])
# unique_channels = unique_channels[unique_channels['Provider/Channel'].notna()]

unique_channels.shape
# df2 = group.apply(lambda x: x['Channel_Country'].unique())

In [ ]:
# for s in df2:
# len(df2)
entity_channels_df = unique_channels[['Provider/Channel','Channel_Country']]

entity_channels_df.shape

In [ ]:
entity_channels_df.to_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/entities/v6_channels.csv',index=False)

In [ ]:
networks_df = all_channels[all_channels['is_Provider'] == True]
# networks_df = all_channels[~all_channels['Provider_URL'].isna()]
networks_df.shape
# len(networks_df['Provider'].unique())

In [ ]:
unique_networks = networks_df.drop_duplicates(subset=['Provider'])

unique_networks.shape

In [ ]:
entity_networks_df = unique_networks[['Provider', 'Provider_Country']]

entity_networks_df

In [ ]:
entity_networks_df.to_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/entities/v6_networks.csv',index=False)

In [ ]:
# channel_instances_df = channel_instances_df[channel_instances_df['Provider/Channel'].notna()]
channel_instances_language = channel_instances_df[['Provider/Channel','Sattelite_Name','freq_alt', 'Languages']]
# channel_instances_df.shape
# channel_instances_language.head(10)


In [ ]:
# channel_instances_df.head(10)

In [ ]:
# channel_instances_df[channel_instances_df['Provider'] == 'BYU TV Global']
# channel_instances_df = channel_instances_df[channel_instances_df['Provider/Channel'].isna()]
# discarded_instances_df = channel_instances_df[~channel_instances_df['Provider/Channel'].notna()]
# channel_instances_df.head(10)

In [ ]:

print(channel_instances_df.shape)
# print(discarded_instances_df.shape)


In [ ]:
channel_instances_df['Languages'].unique()

In [ ]:

# channel_instances_language['Languages']
# Split by each comma
channel_instances_language['Languages'] = channel_instances_language['Languages'].str.split(' ')

channel_instances_language = channel_instances_language.explode('Languages')

In [ ]:
channel_instances_language['Languages'].unique()

In [ ]:
channel_instances_language[channel_instances_language['Provider/Channel'] == 'BYU TV Global']

In [ ]:
channel_instances_language.to_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/entities/v6_channels_instance_language.csv',index=False)


In [ ]:
channel_instances_df.keys()

In [ ]:
channel_instances_encryption = channel_instances_df[['Provider/Channel','Sattelite_Name','freq_alt', 'encryption_alt']]
# channel_instances_encryption['encryption_alt'].value_counts()

In [ ]:

# channel_instances_language = channel_instances_df[['Provider/Channel','Sattelite_Name','freq_alt', 'Languages']]
channel_instances_encryption['encryption_alt'] = channel_instances_encryption['encryption_alt'].str.split(',')

channel_instances_encryption = channel_instances_encryption.explode('encryption_alt')


In [ ]:

channel_instances_encryption

In [ ]:
channel_instances_encryption.to_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/entities/v6_channels_instance_encryption.csv',index=False)


In [ ]:
channel_instances_df = pd.read_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/entities/channel_instances.csv')
# print(channel_instances_df.keys())
# channel_instances_df.shape

# channel_instances_df[['Sattelite_Name','Provider/Channel','FEC']][101:150]


In [ ]:
channel_instances_df = channel_instances_df[channel_instances_df['is_Provider'] == False]

channel_instances_df.shape


In [ ]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# channel_instances_df[channel_instances_df['Sattelite_Name'] == 'Intelsat-18'][34:100]

channel_instances_df.drop_duplicates(subset=['Provider/Channel','Sattelite_Name', 'freq_alt'], inplace=True)
# channel_instances_df['Channel'].isna().sum()


In [ ]:
channel_instances_df.to_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/entities/v6_channel_instances.csv')

In [ ]:
# duplicates_df = channel_instances_df[channel_instances_df.duplicated(subset=['Channel','Sattelite_Name', 'freq_alt'], keep=False)]

# duplicates_df.shape

channel_instances_df['fec_alt'].value_counts()

In [ ]:
for col in duplicates_df.keys():
    # val_cnt = (duplicates_df[col].isna())
    print(f'{col}: {duplicates_df[col].isna().sum()}')
    
    

In [ ]:
sat_html = '''

<html>
<head>
<!-- Google tag (gtag.js) -->
<script async src="https://www.googletagmanager.com/gtag/js?id=G-9M05T6DPQ6"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'G-9M05T6DPQ6');
</script>
<title>Optus D3 at 156.0°E - LyngSat</title>
<meta http-equiv="Content-Type" content="text/html"; charset="iso-8859-1">
<meta name="keywords" content="Optus D3 lyngsat satellite frequency chart transponder mux TV radio Lyngemark analog digital channel MPEG-2 MPEG-4 AVC HEVC DVB DVB-S DVB-S2 DVB-S2X DSS ISDB Digicipher ABS FTA SR FEC SID PID PAL HD UHD Viaccess Conax Irdeto Mediaguard Cryptoworks Videoguard RAS BISS PowerVu free FTA">
<meta name="revisit-after" content="1 days">
<meta name="viewport" content="width=device-width">
<base target="_top">
<link href="misc/general.css" rel="stylesheet" type="text/css">
<link href="misc/responsive.css" rel="stylesheet" type="text/css">
<link href="misc/style.css" rel="stylesheet" type="text/css">
</head>
<body bgcolor='moccasin' text='#000000' link='blue' vlink='blue'>

<div align="center">
<script type="text/javascript" src="//lyngsat.com/system/loadclick.js"></script>
<div class="desktop"></div><div class="tablet"></div><div class="mobile"></div><div class="desktop"></div><div class="desktop">
</div>
<!-- big table start -->
<table class="bigtable" border=0 cellspacing=0 cellpadding=0>
<tr>
<td width="160" align="center" valign="top">
<div class="desktop">
<table border=0 cellspacing=1 cellpadding=1>
<tr><td align='center'><font face='Arial' size=2><a href='http://www.lyngsat.com/advert.html'><font color='blue'><b>Advertisements</b></font></a></font></td></tr><tr align='center'>
<td width=160 height=250>
<table width=160 height=100 border=0 cellspacing=0 cellpadding=4 bgcolor="lemonchiffon">
<tr><td align="left" colspan=2><font face="Arial" size=1 color="black"><i>News at <a href="http://www.lyngsat.com/">LyngSat</a>:</i></td></tr>
<tr><td align="center" width=32 height=40><img src="/logo/tv/cc/christ-revealed-tv-zw.png" width=32 height=24></td><td align="left" width=125><font face="Arial" size=2><a href="http://www.lyngsat.com/tvchannels/zw/Christ-Revealed-TV.html">Christ Revealed TV</a> <font size=1><br>on <a href="https://www.lyngsat.com/Intelsat-20.html">Intelsat 20</font></a></font></td></tr><tr><td align="center" width=32 height=40><img src="/logo/tv/rr/raapchik-in.png" width=32 height=24></td><td align="left" width=125><font face="Arial" size=2><a href="http://www.lyngsat.com/tvchannels/in/Raapchik.html">Raapchik</a> <font size=1><br>on <a href="https://www.lyngsat.com/G-Sat-15.html">G-Sat 15</font></a></font></td></tr><tr><td align="center" width=32 height=40><img src="/logo/tv/zz/zee-action-in.png" width=32 height=24></td><td align="left" width=125><font face="Arial" size=2><a href="http://www.lyngsat.com/tvchannels/in/Zee-Action.html">Zee Action</a> <font size=1><br>on <a href="https://www.lyngsat.com/G-Sat-15.html">G-Sat 15</font></a></font></td></tr><tr><td align="center" width=32 height=40><img src="/logo/tv/nn/news-24-in.png" width=32 height=24></td><td align="left" width=125><font face="Arial" size=2><a href="http://www.lyngsat.com/tvchannels/in/News-24.html">News 24</a> <font size=1><br>on <a href="https://www.lyngsat.com/G-Sat-15.html">G-Sat 15</font></a></font></td></tr><tr><td align="center" width=32 height=40><img src="/logo/tv/pp/pear-tv.png" width=32 height=24></td><td align="left" width=125><font face="Arial" size=2><a href="http://www.lyngsat.com/tvchannels/in/Pear-TV.html">Pear TV</a> <font size=1><br>on <a href="https://www.lyngsat.com/G-Sat-15.html">G-Sat 15</font></a></font></td></tr><tr><td align="center" width=32 height=40><img src="/logo/tv/vv/vtv-gujarati-in.png" width=32 height=24></td><td align="left" width=125><font face="Arial" size=2><a href="http://www.lyngsat.com/tvchannels/in/VTV-Gujarati.html">VTV Gujarati</a> <font size=1><br>on <a href="https://www.lyngsat.com/G-Sat-15.html">G-Sat 15</font></a></font></td></tr></table>
</td>
</tr>
<tr align='center'>
<td width=160 height=250>
<table width=160 height=100 border=0 cellspacing=0 cellpadding=4 bgcolor="lightgreen">
<tr><td align="left" colspan=2><font face="Arial" size=1 color="black"><i>News at <a href="http://www.lyngsat.com/stream/">LyngSat Stream</a>:</i></td></tr>
<tr><td align="center" width=40 height=30><img src="/logo/tv/tt/taiwan-plus-tw.svg" width=40 height=30></td><td align="left" width=115><font face="Arial" size=2><a href="http://www.lyngsat.com/stream/tvchannels/tw/Taiwan--.html">Taiwan +</a></font></td></tr><tr><td align="center" width=40 height=30><img src="/logo/tv/jj/jibek-joly-kz.png" width=40 height=30></td><td align="left" width=115><font face="Arial" size=2><a href="http://www.lyngsat.com/stream/tvchannels/kz/Jibek-Joly.html">Jibek Joly</a></font></td></tr><tr><td align="center" width=40 height=30><img src="/logo/tv/pp/prima-news-ro.svg" width=40 height=30></td><td align="left" width=115><font face="Arial" size=2><a href="http://www.lyngsat.com/stream/tvchannels/ro/Prima-News.html">Prima News</a></font></td></tr><tr><td align="center" width=40 height=30><img src="/logo/tv/aa/almaty-tv-kz.svg" width=40 height=30></td><td align="left" width=115><font face="Arial" size=2><a href="http://www.lyngsat.com/stream/tvchannels/kz/Almaty-TV.html">Almaty TV</a></font></td></tr><tr><td align="center" width=40 height=30><img src="/logo/tv/ss/schwarzrotgold-tv-dz.png" width=40 height=30></td><td align="left" width=115><font face="Arial" size=2><a href="http://www.lyngsat.com/stream/tvchannels/de/SchwarzRotGold-TV.html">SchwarzRotGold TV</a></font></td></tr><tr><td align="center" width=40 height=30><img src="/logo/tv/ww/wedo-amor-ch.png" width=40 height=30></td><td align="left" width=115><font face="Arial" size=2><a href="http://www.lyngsat.com/stream/tvchannels/ch/Wedo-Amor.html">Wedo Amor</a></font></td></tr></table>
</td>
</tr>
</table>
</div>
</td>
<td width=728 align="center" valign="top">
<br>
<table width=720 border=0 cellspacing=1 cellpadding=1>
<tr valign="top">
<td align="center" bordercolor=moccasin bgcolor=moccasin><font face="Arial" size=2>
<a href="https://www.lyngsat.com/"><img src="/images/lyngsat_210x48.png" width=210 height=48 border=0></a>
<br><br>
<b><font face="Arial" size=5>Optus D3 at 156.<font face="Arial" size=3>0</font><font face="Arial" size=5>°E</font></font></b>
</td>
</tr>
<tr valign="baseline">
<td align="center" width=720>
<div class="desktop"><table width=728 border=0 cellspacing=1 cellpadding=1><tr><td align='center' width=728 height=90><a href='https://www.intelsat.com/events/nab-show-2024/?utm_source=display&utm_medium=sponsored_content&utm_campaign=2024_media&utm_content=website&utm_term=lyngsat' target='_blank'><img src='/as10/intelsat_nab_240228.gif' title='Intelsat' width=728 height=90 border=0 onload='LoadAction("Intelsat","ble03d2","intelsat_nab_240228.gif","90","","","","")' onclick='ClickAction("Intelsat","ble03d2","intelsat_nab_240228.gif","90","","","","");'></a></td>
</tr></table></div><div class="tablet"><table width=728 border=0 cellspacing=1 cellpadding=1><tr><td align='center' width=728 height=90><a href='https://www.intelsat.com/events/nab-show-2024/?utm_source=display&utm_medium=sponsored_content&utm_campaign=2024_media&utm_content=website&utm_term=lyngsat' target='_blank'><img src='/as10/intelsat_nab_240228.gif' title='Intelsat' width=728 height=90 border=0 onload='LoadAction("Intelsat","ble03t2","intelsat_nab_240228.gif","90","","","","")' onclick='ClickAction("Intelsat","ble03t2","intelsat_nab_240228.gif","90","","","","");'></a></td>
</tr></table></div><div class="mobile"></div></td>
</tr>
</table>
<table width=720 border=0 cellspacing=0 cellpadding=0>
<tr>
<td><font size=3 color="#ff9C00"><b><img src="/images/pixel_brun.gif" width=720 height=1 vspace=4></b></td>
</tr>
</table>
<table width=720 border=0 cellspacing=0 cellpadding=0 bgcolor="khaki">
<tr align="center">
<td align="left"><font face="Arial" size=2><b><a href="https://www.lyngsat.com/america.html">America</a></b></td>
<td align="center"><font face="Arial" size=2 color="#000000">
<b><a href="https://www.lyngsat.com/index.html">Main</a>
 | <a href="https://www.lyngsat.com/asia.html">Asia</a>
 | <a href="https://www.lyngsat.com/packages/asia.html">Asia <img src="/p.gif" height=15 width=15 align="absmiddle"></a>
 | <a href="https://www.lyngsat.com/tracker/asia.html">Asia <img src="/map-mini.gif" height=15 width=30 align="absmiddle"></a>
 | <a href="https://www.lyngsat.com/uhd/index.html">UHD</a>
 | <a href="https://www.lyngsat.com/headlines.html">Headlines</a>
 | <a href="https://www.lyngsat.com/launches/index.html">Launches</a>
</td>
<td align="right"><font face="Arial" size=2><b><a href="https://www.lyngsat.com/europe.html">Europe</a></b></td>
</tr>
</table>
<table width=720 border=0 cellspacing=0 cellpadding=0>
<tr>
<td><font size=3 color="#ff9C00"><b><img src="/images/pixel_brun.gif" width=720 height=1 vspace=4></b></td>
</tr>
</table>
<table width=720 border=0 cellspacing=0 cellpadding=0 bgcolor="khaki">
<tr align="center">
<td align="left"><font face="Arial" size=2><b><a href="https://www.lyngsat.com/ABS-6.html">159.0°E</a> &lt;C&gt; <a href="https://www.lyngsat.com/JCSAT-2B.html">154.0°E</a></b></td>
<td align="center"><font face="Arial" size=2 color="#000000">
<b><a href="https://www.lyngsat.com/uhd/Optus-D3.html">UHD</a> | <a href="https://www.lyngsat.com/tracker/Optus-D3.html">SatTracker</a> | <a href="https://www.lyngsat.com/maps/Optus-D3.html">LyngSat Maps</a>
</b>
</td>
<td align="right"><font face="Arial" size=2><b><a href="https://www.lyngsat.com/ABS-6.html">159.0°E</a> &lt;Ku&gt; <a href="https://www.lyngsat.com/JCSAT-2B.html">154.0°E</a></b></td>
</tr>
</table>
<table width=720 border=0 cellspacing=0 cellpadding=0>
<tr>
<td><font size=3 color="#ff9C00"><b><img src="/images/pixel_brun.gif" width=720 height=1 vspace=4></b></td>
</tr>
</table>
<table width=720 border=0 cellspacing=0 cellpadding=0 bgcolor="khaki">
<tr align="center">
<td align="center"><font face="Arial" size=2 color="#000000">
<b><a href="https://www.lyngsat.com/Optus-D3-10.html">Optus D3/10</a>
 | <a href="https://www.lyngsat.com/Optus-D3.html">Optus D3</a>
 | <a href="https://www.lyngsat.com/Optus-10.html">Optus 10</a>
</b>
</td>
</tr>
</table>
<table width=720 border=0 cellspacing=0 cellpadding=0>
<tr>
<td><font size=3 color="#ff9C00"><b><img src="/images/pixel_brun.gif" width=720 height=1 vspace=4></b></td>
</tr>
</table>

<script language="Javascript">
<!--
function dummy(){return}
function ifc(f){window.status="IF: "+(5150-f)+" MHz"}
function ifk1(f){window.status="IF: "+(f-10750)+" MHz"}
function ifk2(f){window.status="IF: "+(f-11250)+" MHz"}
//-->
</script>

<table border=0 cellspacing=0 cellpadding=0><tr><td><font face="Verdana" size=2 color="black"><i>This satellite is <font color="red">below horizon</font> in Cairo, Egypt</i></td></tr></table>

<table border=0 cellspacing=0 cellpadding=0><tr><td><font face="Verdana" size=2 color="black"><i>The EIRP values are for Cairo, Egypt</i></td></tr></table>

<br>
<div class="desktab">
<div class="desktop"></div><div class="tablet"><table width=720 height=75 border=0 cellspacing=1 cellpadding=1>
<tr align='center'>
<td width=50 height=75 valign='center'><a href='https://sat.tv/boxes/daly-export?utm_source=Lyngsat&utm_medium=Banner&utm_campaign=Sales&utm_id=Lyngsat+Banner+2' target='_blank'><img src='/as10/sat-tv_en_eg_240112.gif' title='Sat.tv Daly Export' width=675 height=75 border=0 onload='LoadAction("Sattv","sat02t4enega","sat-tv_en_eg_240112.gif","75","2","50","H","")' onclick='ClickAction("Sattv","sat02t4enega","sat-tv_en_eg_240112.gif","75","2","50","H","");'></a></td>
</tr>
</table>
</div><div class="mobile"></div><br>
</div>
<table width=720 border cellspacing=0 cellpadding=0>
<tr>
<td class="td-copyright" colspan=10 align="center"><font face="Arial" size=2>Optus D3 &#169; LyngSat, last updated 2024-04-02 - https://www.lyngsat.com/Optus-D3.html</td>
</tr>
<tr>
<td width=70 align="center"><font face="Verdana"><font size=2>Frequency</font><font size=1><br>Beam<br>EIRP (dBW)</font></td>
<td width=60 align="center"><font face="Verdana"><font size=1>System<br>SR<br>FEC</font></td>
<td width=52 align="center"><font face="Verdana"><font size=1>Logo<br>SID</font></td>
<td width=200 align="left"><font face="Verdana" size=2><i>Provider Name</i><br>Channel Name</font></td>
<td width=15><br></td>
<td width=70 align="left"><font face="Verdana" size=1><font color="blue">ONID-TID</font><br>Compression<br>Format</font></td>
<td width=50 align="center"><font face="Verdana" size=1>VPID</font></td>
<td width=50 align="left"><font face="Verdana" size=1><font color="blue">C/N lock</font><br>Audio</font></td>
<td width=60 align="left"><font face="Verdana" size=1>Encryption</font></td>
<td width=80 align="left"><font face="Verdana" size=1>Source<br>Updated</font></td>
</tr>
<tr>
<td style="background:linear-gradient(to right,#ffb6c1 50%,#bbffbb 51%);" align="center"><font face="Verdana"><font size=2><b><a href="https://www.lyngsat.com/muxes/Optus-D3_Australia_11720-V.html" style="color:blue">11720&nbsp;V</a></b><br><font size=1>tp B1</font><br></font><font size=1><a href="https://www.lyngsat.com/maps/footprints/Optus-D3-Australia.html">Australia</a><br>0</font></td>
<td style="background:#bbffbb" align="center"><font face="Verdana" size=1>DVB-S2<br>8PSK<br>30000<br>3/5</font></td>
<td style="background:#ffb6c1">
<img src="/logo/corp/vv/vast_au.png" style=width:52;height:39; border=0>
</td>
<td style="background:#ffb6c1"><font face="Arial"><font size=2><b><i><a href="https://www.lyngsat.com/packages/VAST.html">VAST</a></i></b></td>
<td style="background:#ffb6c1"><a href="https://www.lyngsat.com/packages/VAST.html"><img src="/p.gif" title="Packages" height=15 width=15 align=absbottom border=1></a><br></td>
<td style="background:#ffb6c1"></td>
<td align="center" style="background:#ffb6c1"><font face="Verdana" size=1></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1><font color="blue">5.5</font>&nbsp;<br></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1>Irdeto</font></td>
<td><font face="Verdana" size=1>C Sutton<br>221214</font></td>
</tr>
<tr>
<td style="background:linear-gradient(to right,#ffb6c1 50%,#bbffbb 51%);" align="center"><font face="Verdana"><font size=2><b><a href="https://www.lyngsat.com/muxes/Optus-D3_Australia_11720-H.html" style="color:blue">11720&nbsp;H</a></b><br><font size=1>tp B13</font><br></font><font size=1><a href="https://www.lyngsat.com/maps/footprints/Optus-D3-Australia.html">Australia</a><br>0</font></td>
<td style="background:#bbffbb" align="center"><font face="Verdana" size=1>DVB-S2<br>8PSK<br>29450<br>3/5</font></td>
<td style="background:#ffb6c1">
<img src="/logo/corp/ff/foxtel-au.svg" style=width:52;height:39; border=0>
</td>
<td style="background:#ffb6c1"><font face="Arial"><font size=2><b><i><a href="https://www.lyngsat.com/packages/Foxtel.html">Foxtel</a></i></b></td>
<td style="background:#ffb6c1"><a href="https://www.lyngsat.com/packages/Foxtel.html"><img src="/p.gif" title="Packages" height=15 width=15 align=absbottom border=1></a><br></td>
<td style="background:#ffb6c1"></td>
<td align="center" style="background:#ffb6c1"><font face="Verdana" size=1></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1><font color="blue">5.5</font>&nbsp;<br></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1>Irdeto</font></td>
<td><font face="Verdana" size=1>C Sutton<br>220401</font></td>
</tr>
<tr>
<td style="background:linear-gradient(to right,#ffb6c1 50%,#bbffbb 51%);" align="center"><font face="Verdana"><font size=2><b><a href="https://www.lyngsat.com/muxes/Optus-D3_Australia_11762-H.html" style="color:blue">11762&nbsp;H</a></b><br><font size=1>tp B14</font><br></font><font size=1><a href="https://www.lyngsat.com/maps/footprints/Optus-D3-Australia.html">Australia</a><br>0</font></td>
<td style="background:#bbffbb" align="center"><font face="Verdana" size=1>DVB-S2<br>8PSK<br>29450<br>3/5</font></td>
<td style="background:#ffb6c1">
<img src="/logo/corp/ff/foxtel-au.svg" style=width:52;height:39; border=0>
</td>
<td style="background:#ffb6c1"><font face="Arial"><font size=2><b><i><a href="https://www.lyngsat.com/packages/Foxtel.html">Foxtel</a></i></b></td>
<td style="background:#ffb6c1"><a href="https://www.lyngsat.com/packages/Foxtel.html"><img src="/p.gif" title="Packages" height=15 width=15 align=absbottom border=1></a><br></td>
<td style="background:#ffb6c1"> <font face="Verdana" size=1><font color="blue">4096-14</font></font><br></td>
<td align="center" style="background:#ffb6c1"><font face="Verdana" size=1></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1><font color="blue">5.5</font>&nbsp;<br></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1>Irdeto<br>Videoguard</font></td>
<td><font face="Verdana" size=1>C Sutton<br>230301</font></td>
</tr>
<tr>
<td style="background:linear-gradient(to right,#ffb6c1 50%,#bbffbb 51%);" align="center"><font face="Verdana"><font size=2><b><a href="https://www.lyngsat.com/muxes/Optus-D3_Australia_11804-V.html" style="color:blue">11804&nbsp;V</a></b><br><font size=1>tp B3</font><br></font><font size=1><a href="https://www.lyngsat.com/maps/footprints/Optus-D3-Australia.html">Australia</a><br>0</font></td>
<td style="background:#bbffbb" align="center"><font face="Verdana" size=1>DVB-S2<br>8PSK<br>30000<br>3/5</font></td>
<td style="background:#ffb6c1">
<img src="/logo/corp/vv/vast_au.png" style=width:52;height:39; border=0>
</td>
<td style="background:#ffb6c1"><font face="Arial"><font size=2><b><i><a href="https://www.lyngsat.com/packages/VAST.html">VAST</a></i></b></td>
<td style="background:#ffb6c1"><a href="https://www.lyngsat.com/packages/VAST.html"><img src="/p.gif" title="Packages" height=15 width=15 align=absbottom border=1></a><br></td>
<td style="background:#ffb6c1"> <font face="Verdana" size=1><font color="blue">?-4</font></font><br></td>
<td align="center" style="background:#ffb6c1"><font face="Verdana" size=1></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1><font color="blue">5.5</font>&nbsp;<br></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1>Irdeto</font></td>
<td><font face="Verdana" size=1>C Sutton<br>231226</font></td>
</tr>
<tr>
<td style="background:linear-gradient(to right,#ffb6c1 50%,#bbffbb 51%);" align="center"><font face="Verdana"><font size=2><b><a href="https://www.lyngsat.com/muxes/Optus-D3_Australia_11804-H.html" style="color:blue">11804&nbsp;H</a></b><br><font size=1>tp B15</font><br></font><font size=1><a href="https://www.lyngsat.com/maps/footprints/Optus-D3-Australia.html">Australia</a><br>0</font></td>
<td style="background:#bbffbb" align="center"><font face="Verdana" size=1>DVB-S2<br>8PSK<br>29450<br>3/5</font></td>
<td style="background:#ffb6c1">
<img src="/logo/corp/ff/foxtel-au.svg" style=width:52;height:39; border=0>
</td>
<td style="background:#ffb6c1"><font face="Arial"><font size=2><b><i><a href="https://www.lyngsat.com/packages/Foxtel.html">Foxtel</a></i></b></td>
<td style="background:#ffb6c1"><a href="https://www.lyngsat.com/packages/Foxtel.html"><img src="/p.gif" title="Packages" height=15 width=15 align=absbottom border=1></a><br></td>
<td style="background:#ffb6c1"> <font face="Verdana" size=1><font color="blue">?-26</font></font><br></td>
<td align="center" style="background:#ffb6c1"><font face="Verdana" size=1></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1><font color="blue">5.5</font>&nbsp;<br></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1>Irdeto<br>Videoguard</font></td>
<td><font face="Verdana" size=1>C Sutton<br>230706</font></td>
</tr>
<tr>
<td style="background:linear-gradient(to right,#ffb6c1 50%,#bbffbb 51%);" align="center"><font face="Verdana"><font size=2><b><a href="https://www.lyngsat.com/muxes/Optus-D3_Australia_11845-V.html" style="color:blue">11845&nbsp;V</a></b><br><font size=1>tp B4</font><br></font><font size=1><a href="https://www.lyngsat.com/maps/footprints/Optus-D3-Australia.html">Australia</a><br>0</font></td>
<td style="background:#ffffbb" align="center"><font face="Verdana" size=1>DVB-S<br>27800<br>3/4</font></td>
<td style="background:#ffb6c1">
<img src="/logo/corp/ff/foxtel-au.svg" style=width:52;height:39; border=0>
</td>
<td style="background:#ffb6c1"><font face="Arial"><font size=2><b><i><a href="https://www.lyngsat.com/packages/Foxtel.html">Foxtel</a></i></b></td>
<td style="background:#ffb6c1"><a href="https://www.lyngsat.com/packages/Foxtel.html"><img src="/p.gif" title="Packages" height=15 width=15 align=absbottom border=1></a><br></td>
<td style="background:#ffb6c1"> <font face="Verdana" size=1><font color="blue">?-35</font></font><br></td>
<td align="center" style="background:#ffb6c1"><font face="Verdana" size=1></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1><font color="blue">5.5</font>&nbsp;<br></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1>Irdeto<br>Videoguard</font></td>
<td><font face="Verdana" size=1>C Sutton<br>211018</font></td>
</tr>
<tr>
<td style="background:linear-gradient(to right,#ffb6c1 50%,#bbffbb 51%);" align="center"><font face="Verdana"><font size=2><b><a href="https://www.lyngsat.com/muxes/Optus-D3_Australia_11845-H.html" style="color:blue">11845&nbsp;H</a></b><br><font size=1>tp B16</font><br></font><font size=1><a href="https://www.lyngsat.com/maps/footprints/Optus-D3-Australia.html">Australia</a><br>0</font></td>
<td style="background:#bbffbb" align="center"><font face="Verdana" size=1>DVB-S2<br>8PSK<br>29450<br>3/5</font></td>
<td style="background:#ffb6c1">
<img src="/logo/corp/ff/foxtel-au.svg" style=width:52;height:39; border=0>
</td>
<td style="background:#ffb6c1"><font face="Arial"><font size=2><b><i><a href="https://www.lyngsat.com/packages/Foxtel.html">Foxtel</a></i></b></td>
<td style="background:#ffb6c1"><a href="https://www.lyngsat.com/packages/Foxtel.html"><img src="/p.gif" title="Packages" height=15 width=15 align=absbottom border=1></a><br></td>
<td style="background:#ffb6c1"> <font face="Verdana" size=1><font color="blue">?-24</font></font><br></td>
<td align="center" style="background:#ffb6c1"><font face="Verdana" size=1></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1><font color="blue">5.5</font>&nbsp;<br></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1>Irdeto<br>Videoguard</font></td>
<td><font face="Verdana" size=1>C Sutton<br>230705</font></td>
</tr>
<tr>
<td style="background:linear-gradient(to right,#ffb6c1 50%,#bbffbb 51%);" align="center"><font face="Verdana"><font size=2><b><a href="https://www.lyngsat.com/muxes/Optus-D3_Australia_11886-V.html" style="color:blue">11886&nbsp;V</a></b><br><font size=1>tp B5</font><br></font><font size=1><a href="https://www.lyngsat.com/maps/footprints/Optus-D3-Australia.html">Australia</a><br>0</font></td>
<td style="background:#bbffbb" align="center"><font face="Verdana" size=1>DVB-S2<br>8PSK<br>30000<br>3/5</font></td>
<td style="background:#ffb6c1">
<img src="/logo/corp/vv/vast_au.png" style=width:52;height:39; border=0>
</td>
<td style="background:#ffb6c1"><font face="Arial"><font size=2><b><i><a href="https://www.lyngsat.com/packages/VAST.html">VAST</a></i></b></td>
<td style="background:#ffb6c1"><a href="https://www.lyngsat.com/packages/VAST.html"><img src="/p.gif" title="Packages" height=15 width=15 align=absbottom border=1></a><br></td>
<td style="background:#ffb6c1"> <font face="Verdana" size=1><font color="blue">?-2</font></font><br></td>
<td align="center" style="background:#ffb6c1"><font face="Verdana" size=1></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1><font color="blue">5.5</font>&nbsp;<br></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1>Irdeto</font></td>
<td><font face="Verdana" size=1>C Sutton<br>220523</font></td>
</tr>
<tr>
<td style="background:linear-gradient(to right,#ffb6c1 50%,#bbffbb 51%);" align="center"><font face="Verdana"><font size=2><b><a href="https://www.lyngsat.com/muxes/Optus-D3_Australia_11886-H.html" style="color:blue">11886&nbsp;H</a></b><br><font size=1>tp B17</font><br></font><font size=1><a href="https://www.lyngsat.com/maps/footprints/Optus-D3-Australia.html">Australia</a><br>0</font></td>
<td style="background:#ffffbb" align="center"><font face="Verdana" size=1>DVB-S<br>27800<br>3/4</font></td>
<td style="background:#ffb6c1">
<img src="/logo/corp/ff/foxtel-au.svg" style=width:52;height:39; border=0>
</td>
<td style="background:#ffb6c1"><font face="Arial"><font size=2><b><i><a href="https://www.lyngsat.com/packages/Foxtel.html">Foxtel</a></i></b></td>
<td style="background:#ffb6c1"><a href="https://www.lyngsat.com/packages/Foxtel.html"><img src="/p.gif" title="Packages" height=15 width=15 align=absbottom border=1></a><br></td>
<td style="background:#ffb6c1"> <font face="Verdana" size=1><font color="blue">?-7</font></font><br></td>
<td align="center" style="background:#ffb6c1"><font face="Verdana" size=1></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1><font color="blue">5.5</font>&nbsp;<br></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1>Irdeto<br>Videoguard</font></td>
<td><font face="Verdana" size=1>D de Jong<br>200629</font></td>
</tr>
<tr>
<td rowspan=2 style="background:linear-gradient(to right,#ffb6c1 50%,#bbffbb 51%);" align="center"><font face="Verdana"><font size=2><b><a href="https://www.lyngsat.com/muxes/Optus-D3_Australia_11928-V.html" style="color:blue">11928&nbsp;V</a></b><br><font size=1>tp B6</font><br></font><font size=1><a href="https://www.lyngsat.com/maps/footprints/Optus-D3-Australia.html">Australia</a><br>0</font></td>
<td rowspan=2 style="background:#bbffbb" align="center"><font face="Verdana" size=1>DVB-S2<br>QPSK<br>30000<br>3/5</font></td>
<td style="background:#ffb6c1">
<img src="/logo/corp/vv/vast_au.png" style=width:52;height:39; border=0>
</td>
<td style="background:#ffb6c1"><font face="Arial"><font size=2><b><i><a href="https://www.lyngsat.com/packages/VAST.html">VAST</a></i></b></td>
<td style="background:#ffb6c1"><a href="https://www.lyngsat.com/packages/VAST.html"><img src="/p.gif" title="Packages" height=15 width=15 align=absbottom border=1></a><br></td>
<td style="background:#ffb6c1"> <font face="Verdana" size=1><font color="blue">4095-7</font></font><br></td>
<td align="center" style="background:#ffb6c1"><font face="Verdana" size=1></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1><font color="blue">2.2</font>&nbsp;<br></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1>Irdeto</font></td>
<td><font face="Verdana" size=1>C Sutton<br>211019</font></td>
</tr>
<tr>
<td style="background:#ffffbb" align="center" width=52><font size=1 face="Verdana">&nbsp;&nbsp;&nbsp;11</font></td>
<td style="background:#ffffbb"><font face="Arial"><font size=2 color="gray"><b>[Tune Channel test card]</b></td>
<td style="background:#ffffbb"><br></td>
<td style="background:#ffffbb"><font face="Verdana" size=1>MPEG-2/SD</td>
<td align="center" style="background:#ffffbb"><font face="Verdana" size=1>&nbsp;801</font></td>
<td style="background:#ffffbb"><font face="Verdana" size=1>&nbsp;802&nbsp;<br></font></td>
<td style="background:#ffffbb"><font face="Verdana" size=1></font></td>
<td></td>
</tr>
<tr>
<td style="background:linear-gradient(to right,#ffb6c1 50%,#bbffbb 51%);" align="center"><font face="Verdana"><font size=2><b><a href="https://www.lyngsat.com/muxes/Optus-D3_Australia_11928-H.html" style="color:blue">11928&nbsp;H</a></b><br><font size=1>tp B18</font><br></font><font size=1><a href="https://www.lyngsat.com/maps/footprints/Optus-D3-Australia.html">Australia</a><br>0</font></td>
<td style="background:#ffffbb" align="center"><font face="Verdana" size=1>DVB-S<br>27800<br>3/4</font></td>
<td style="background:#ffb6c1">
<img src="/logo/corp/ff/foxtel-au.svg" style=width:52;height:39; border=0>
</td>
<td style="background:#ffb6c1"><font face="Arial"><font size=2><b><i><a href="https://www.lyngsat.com/packages/Foxtel.html">Foxtel</a></i></b></td>
<td style="background:#ffb6c1"><a href="https://www.lyngsat.com/packages/Foxtel.html"><img src="/p.gif" title="Packages" height=15 width=15 align=absbottom border=1></a><br></td>
<td style="background:#ffb6c1"> <font face="Verdana" size=1><font color="blue">?-11</font></font><br></td>
<td align="center" style="background:#ffb6c1"><font face="Verdana" size=1></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1><font color="blue">5.5</font>&nbsp;<br></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1>Irdeto<br>Videoguard</font></td>
<td><font face="Verdana" size=1>C Sutton<br>221213</font></td>
</tr>
<tr>
<td style="background:linear-gradient(to right,#ffb6c1 50%,#bbffbb 51%);" align="center"><font face="Verdana"><font size=2><b><a href="https://www.lyngsat.com/muxes/Optus-D3_Australia_11970-H.html" style="color:blue">11970&nbsp;H</a></b><br><font size=1>tp B19</font><br></font><font size=1><a href="https://www.lyngsat.com/maps/footprints/Optus-D3-Australia.html">Australia</a><br>0</font></td>
<td style="background:#ffffbb" align="center"><font face="Verdana" size=1>DVB-S<br>27800<br>3/4</font></td>
<td style="background:#ffb6c1">
<img src="/logo/corp/ff/foxtel-au.svg" style=width:52;height:39; border=0>
</td>
<td style="background:#ffb6c1"><font face="Arial"><font size=2><b><i><a href="https://www.lyngsat.com/packages/Foxtel.html">Foxtel</a></i></b></td>
<td style="background:#ffb6c1"><a href="https://www.lyngsat.com/packages/Foxtel.html"><img src="/p.gif" title="Packages" height=15 width=15 align=absbottom border=1></a><br></td>
<td style="background:#ffb6c1"> <font face="Verdana" size=1><font color="blue">?-18</font></font><br></td>
<td align="center" style="background:#ffb6c1"><font face="Verdana" size=1></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1><font color="blue">5.5</font>&nbsp;<br></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1>Irdeto<br>Videoguard</font></td>
<td><font face="Verdana" size=1>C Sutton<br>230301</font></td>
</tr>
<tr>
<td style="background:linear-gradient(to right,#ffb6c1 50%,#bbffbb 51%);" align="center"><font face="Verdana"><font size=2><b><a href="https://www.lyngsat.com/muxes/Optus-D3_Australia_12011-H.html" style="color:blue">12011&nbsp;H</a></b><br><font size=1>tp B20</font><br></font><font size=1><a href="https://www.lyngsat.com/maps/footprints/Optus-D3-Australia.html">Australia</a><br>0</font></td>
<td style="background:#bbffbb" align="center"><font face="Verdana" size=1>DVB-S2<br>8PSK<br>29455<br>3/5</font></td>
<td style="background:#ffb6c1">
<img src="/logo/corp/ff/foxtel-au.svg" style=width:52;height:39; border=0>
</td>
<td style="background:#ffb6c1"><font face="Arial"><font size=2><b><i><a href="https://www.lyngsat.com/packages/Foxtel.html">Foxtel</a></i></b></td>
<td style="background:#ffb6c1"><a href="https://www.lyngsat.com/packages/Foxtel.html"><img src="/p.gif" title="Packages" height=15 width=15 align=absbottom border=1></a><br></td>
<td style="background:#ffb6c1"> <font face="Verdana" size=1><font color="blue">?-20</font></font><br></td>
<td align="center" style="background:#ffb6c1"><font face="Verdana" size=1></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1><font color="blue">5.5</font>&nbsp;<br></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1>Irdeto<br>Videoguard</font></td>
<td><font face="Verdana" size=1>D de Jong<br>200629</font></td>
</tr>
<tr>
<td style="background:linear-gradient(to right,#ffb6c1 50%,#bbffbb 51%);" align="center"><font face="Verdana"><font size=2><b><a href="https://www.lyngsat.com/muxes/Optus-D3_Australia_12052-V.html" style="color:blue">12052&nbsp;V</a></b><br><font size=1>tp B9</font><br></font><font size=1><a href="https://www.lyngsat.com/maps/footprints/Optus-D3-Australia.html">Australia</a><br>0</font></td>
<td style="background:#ffffbb" align="center"><font face="Verdana" size=1>DVB-S<br>27800<br>3/4</font></td>
<td style="background:#ffb6c1">
<img src="/logo/corp/ff/foxtel-au.svg" style=width:52;height:39; border=0>
</td>
<td style="background:#ffb6c1"><font face="Arial"><font size=2><b><i><a href="https://www.lyngsat.com/packages/Foxtel.html">Foxtel</a></i></b></td>
<td style="background:#ffb6c1"><a href="https://www.lyngsat.com/packages/Foxtel.html"><img src="/p.gif" title="Packages" height=15 width=15 align=absbottom border=1></a><br></td>
<td style="background:#ffb6c1"></td>
<td align="center" style="background:#ffb6c1"><font face="Verdana" size=1></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1><font color="blue">5.5</font>&nbsp;<br></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1>Irdeto</font></td>
<td><font face="Verdana" size=1>B Gordon<br>221103</font></td>
</tr>
<tr>
<td style="background:linear-gradient(to right,#ffb6c1 50%,#bbffbb 51%);" align="center"><font face="Verdana"><font size=2><b><a href="https://www.lyngsat.com/muxes/Optus-D3_Australia_12052-H.html" style="color:blue">12052&nbsp;H</a></b><br><font size=1>tp B21</font><br></font><font size=1><a href="https://www.lyngsat.com/maps/footprints/Optus-D3-Australia.html">Australia</a><br>0</font></td>
<td style="background:#bbffbb" align="center"><font face="Verdana" size=1>DVB-S2<br>8PSK<br>29450<br>3/5</font></td>
<td style="background:#ffb6c1">
<img src="/logo/corp/ff/foxtel-au.svg" style=width:52;height:39; border=0>
</td>
<td style="background:#ffb6c1"><font face="Arial"><font size=2><b><i><a href="https://www.lyngsat.com/packages/Foxtel.html">Foxtel</a></i></b></td>
<td style="background:#ffb6c1"><a href="https://www.lyngsat.com/packages/Foxtel.html"><img src="/p.gif" title="Packages" height=15 width=15 align=absbottom border=1></a><br></td>
<td style="background:#ffb6c1"> <font face="Verdana" size=1><font color="blue">?-13</font></font><br></td>
<td align="center" style="background:#ffb6c1"><font face="Verdana" size=1></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1><font color="blue">5.5</font>&nbsp;<br></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1>Irdeto<br>Videoguard</font></td>
<td><font face="Verdana" size=1>C Sutton<br>170930</font></td>
</tr>
<tr>
<td style="background:linear-gradient(to right,#ffb6c1 50%,#bbffbb 51%);" align="center"><font face="Verdana"><font size=2><b><a href="https://www.lyngsat.com/muxes/Optus-D3_Australia_12094-V.html" style="color:blue">12094&nbsp;V</a></b><br><font size=1>tp B10</font><br></font><font size=1><a href="https://www.lyngsat.com/maps/footprints/Optus-D3-Australia.html">Australia</a><br>0</font></td>
<td style="background:#ffffbb" align="center"><font face="Verdana" size=1>DVB-S<br>27800<br>3/4</font></td>
<td style="background:#ffb6c1">
<img src="/logo/corp/ff/foxtel-au.svg" style=width:52;height:39; border=0>
</td>
<td style="background:#ffb6c1"><font face="Arial"><font size=2><b><i><a href="https://www.lyngsat.com/packages/Foxtel.html">Foxtel</a></i></b></td>
<td style="background:#ffb6c1"><a href="https://www.lyngsat.com/packages/Foxtel.html"><img src="/p.gif" title="Packages" height=15 width=15 align=absbottom border=1></a><br></td>
<td style="background:#ffb6c1"> <font face="Verdana" size=1><font color="blue">?-31</font></font><br></td>
<td align="center" style="background:#ffb6c1"><font face="Verdana" size=1></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1><font color="blue">5.5</font>&nbsp;<br></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1>Videoguard</font></td>
<td><font face="Verdana" size=1>T Baxter<br>180528</font></td>
</tr>
<tr>
<td style="background:linear-gradient(to right,#ffb6c1 50%,#bbffbb 51%);" align="center"><font face="Verdana"><font size=2><b><a href="https://www.lyngsat.com/muxes/Optus-D3_Australia_12094-H.html" style="color:blue">12094&nbsp;H</a></b><br><font size=1>tp B22</font><br></font><font size=1><a href="https://www.lyngsat.com/maps/footprints/Optus-D3-Australia.html">Australia</a><br>0</font></td>
<td style="background:#bbffbb" align="center"><font face="Verdana" size=1>DVB-S2<br>8PSK<br>29450<br>3/5</font></td>
<td style="background:#ffb6c1">
<img src="/logo/corp/ff/foxtel-au.svg" style=width:52;height:39; border=0>
</td>
<td style="background:#ffb6c1"><font face="Arial"><font size=2><b><i><a href="https://www.lyngsat.com/packages/Foxtel.html">Foxtel</a></i></b></td>
<td style="background:#ffb6c1"><a href="https://www.lyngsat.com/packages/Foxtel.html"><img src="/p.gif" title="Packages" height=15 width=15 align=absbottom border=1></a><br></td>
<td style="background:#ffb6c1"> <font face="Verdana" size=1><font color="blue">?-25</font></font><br></td>
<td align="center" style="background:#ffb6c1"><font face="Verdana" size=1></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1><font color="blue">5.5</font>&nbsp;<br></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1>Videoguard</font></td>
<td><font face="Verdana" size=1>C Sutton<br>230301</font></td>
</tr>
<tr>
<td class="td-copyright" colspan=11 align="center"><font face="Arial" size=2>Optus D3 &#169; LyngSat, last updated 2024-04-02 - https://www.lyngsat.com/Optus-D3.html</td>
</tr>
</table>
<br>
<div class="desktab">
<div class="desktop"></div><div class="tablet"><table width=720 height=75 border=0 cellspacing=1 cellpadding=1>
<tr align='center'>
<td width=50 height=75 valign='center'><a href='https://sat.tv/boxes/maestro?utm_source=Lyngsat&utm_medium=Banner&utm_campaign=Sales&utm_id=Lyngsat+Banner+3' target='_blank'><img src='/as10/sat-tv_en_eg_240112.gif' title='Sat.tv Maestro' width=675 height=75 border=0 onload='LoadAction("Sattv","sat02t4xxegb","sat-tv_en_eg_240112.gif","75","2","50","H","")' onclick='ClickAction("Sattv","sat02t4xxegb","sat-tv_en_eg_240112.gif","75","2","50","H","");'></a></td>
</tr>
</table>
</div><div class="mobile"></div><br>
</div>
<table width=720 border cellspacing=0 cellpadding=0>
<tr>
<td class="td-copyright" colspan=10 align="center"><font face="Arial" size=2>Optus D3 &#169; LyngSat, last updated 2024-04-02 - https://www.lyngsat.com/Optus-D3.html</td>
</tr>
<tr>
<td width=70 align="center"><font face="Verdana"><font size=2>Frequency</font><font size=1><br>Beam<br>EIRP (dBW)</font></td>
<td width=60 align="center"><font face="Verdana"><font size=1>System<br>SR<br>FEC</font></td>
<td width=52 align="center"><font face="Verdana"><font size=1>Logo<br>SID</font></td>
<td width=200 align="left"><font face="Verdana" size=2><i>Provider Name</i><br>Channel Name</font></td>
<td width=15><br></td>
<td width=70 align="left"><font face="Verdana" size=1><font color="blue">ONID-TID</font><br>Compression<br>Format</font></td>
<td width=50 align="center"><font face="Verdana" size=1>VPID</font></td>
<td width=50 align="left"><font face="Verdana" size=1><font color="blue">C/N lock</font><br>Audio</font></td>
<td width=60 align="left"><font face="Verdana" size=1>Encryption</font></td>
<td width=80 align="left"><font face="Verdana" size=1>Source<br>Updated</font></td>
</tr>
<tr>
<td style="background:linear-gradient(to right,#ffb6c1 50%,#bbffbb 51%);" align="center"><font face="Verdana"><font size=2><b><a href="https://www.lyngsat.com/muxes/Optus-D3_Australia_12136-V.html" style="color:blue">12136&nbsp;V</a></b><br><font size=1>tp B11</font><br></font><font size=1><a href="https://www.lyngsat.com/maps/footprints/Optus-D3-Australia.html">Australia</a><br>0</font></td>
<td style="background:#ffffbb" align="center"><font face="Verdana" size=1>DVB-S<br>27800<br>3/4</font></td>
<td style="background:#ffb6c1">
<img src="/logo/corp/ff/foxtel-au.svg" style=width:52;height:39; border=0>
</td>
<td style="background:#ffb6c1"><font face="Arial"><font size=2><b><i><a href="https://www.lyngsat.com/packages/Foxtel.html">Foxtel</a></i></b></td>
<td style="background:#ffb6c1"><a href="https://www.lyngsat.com/packages/Foxtel.html"><img src="/p.gif" title="Packages" height=15 width=15 align=absbottom border=1></a><br></td>
<td style="background:#ffb6c1"> <font face="Verdana" size=1><font color="blue">4096-32</font></font><br></td>
<td align="center" style="background:#ffb6c1"><font face="Verdana" size=1></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1><font color="blue">5.5</font>&nbsp;<br></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1>Irdeto<br>Videoguard</font></td>
<td><font face="Verdana" size=1>C Sutton<br>230705</font></td>
</tr>
<tr>
<td style="background:linear-gradient(to right,#ffb6c1 50%,#bbffbb 51%);" align="center"><font face="Verdana"><font size=2><b><a href="https://www.lyngsat.com/muxes/Optus-D3_Australia_12136-H.html" style="color:blue">12136&nbsp;H</a></b><br><font size=1>tp B23</font><br></font><font size=1><a href="https://www.lyngsat.com/maps/footprints/Optus-D3-Australia.html">Australia</a><br>0</font></td>
<td style="background:#bbffbb" align="center"><font face="Verdana" size=1>DVB-S2<br>8PSK<br>30000<br>3/5</font></td>
<td style="background:#ffb6c1">
<img src="/logo/corp/ff/foxtel-au.svg" style=width:52;height:39; border=0>
</td>
<td style="background:#ffb6c1"><font face="Arial"><font size=2><b><i><a href="https://www.lyngsat.com/packages/Foxtel.html">Foxtel</a></i></b></td>
<td style="background:#ffb6c1"><a href="https://www.lyngsat.com/packages/Foxtel.html"><img src="/p.gif" title="Packages" height=15 width=15 align=absbottom border=1></a><br></td>
<td style="background:#ffb6c1"> <font face="Verdana" size=1><font color="blue">?-9</font></font><br></td>
<td align="center" style="background:#ffb6c1"><font face="Verdana" size=1></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1><font color="blue">5.5</font>&nbsp;<br></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1>Irdeto</font></td>
<td><font face="Verdana" size=1>C Sutton<br>220221</font></td>
</tr>
<tr>
<td style="background:linear-gradient(to right,#ffb6c1 50%,#bbffbb 51%);" align="center"><font face="Verdana"><font size=2><b><a href="https://www.lyngsat.com/muxes/Optus-D3_Australia_12177-H.html" style="color:blue">12177&nbsp;H</a></b><br><font size=1>tp B24</font><br></font><font size=1><a href="https://www.lyngsat.com/maps/footprints/Optus-D3-Australia.html">Australia</a><br>0</font></td>
<td style="background:#ffffbb" align="center"><font face="Verdana" size=1>DVB-S<br>27800<br>3/4</font></td>
<td style="background:#ffb6c1">
<img src="/logo/corp/ff/foxtel-au.svg" style=width:52;height:39; border=0>
</td>
<td style="background:#ffb6c1"><font face="Arial"><font size=2><b><i><a href="https://www.lyngsat.com/packages/Foxtel.html">Foxtel</a></i></b></td>
<td style="background:#ffb6c1"><a href="https://www.lyngsat.com/packages/Foxtel.html"><img src="/p.gif" title="Packages" height=15 width=15 align=absbottom border=1></a><br></td>
<td style="background:#ffb6c1"> <font face="Verdana" size=1><font color="blue">?-10</font></font><br></td>
<td align="center" style="background:#ffb6c1"><font face="Verdana" size=1></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1><font color="blue">5.5</font>&nbsp;<br></font></td>
<td style="background:#ffb6c1"><font face="Verdana" size=1>Irdeto<br>Videoguard</font></td>
<td><font face="Verdana" size=1>C Sutton<br>230705</font></td>
</tr>
<tr>
<td class="td-copyright" colspan=11 align="center"><font face="Arial" size=2>Optus D3 &#169; LyngSat, last updated 2024-04-02 - https://www.lyngsat.com/Optus-D3.html</td>
</tr>
</table>
<br>
<div class="desktab">
<div class="desktop"></div><div class="tablet"><table width=720 height=75 border=0 cellspacing=1 cellpadding=1>
<tr align='center'>
<td width=50 height=75>
<table width=175 height=75 border=0 cellspacing=0 cellpadding=2 bgcolor="lightgreen">
<tr><td align="left" colspan=2><font face="Arial" size=1 color="black"><i>News at <a href="http://www.lyngsat.com/stream/">LyngSat Stream</a>:</i></td></tr>
<tr><td align="center" width=32 height=32><img src="/logo/tv/tt/taiwan-plus-tw.svg" width=32 height=24></td><td align="left" width=175><font face="Arial" size=2><a href="http://www.lyngsat.com/stream/tvchannels/tw/Taiwan--.html">Taiwan +</a></font></td></tr><tr><td align="center" width=32 height=32><img src="/logo/tv/jj/jibek-joly-kz.png" width=32 height=24></td><td align="left" width=175><font face="Arial" size=2><a href="http://www.lyngsat.com/stream/tvchannels/kz/Jibek-Joly.html">Jibek Joly</a></font></td></tr></table>
</td>
<td width=50 height=75>
<table width=175 height=75 border=0 cellspacing=0 cellpadding=2 bgcolor="lemonchiffon">
<tr><td align="left" colspan=2><font face="Arial" size=1 color="black"><i>News at <a href="http://www.lyngsat.com/">LyngSat</a>:</i></td></tr>
<tr><td align="center" width=32 height=32><img src="/logo/tv/cc/christ-revealed-tv-zw.png" width=32 height=24></td><td align="left" width=175><font face="Arial" size=2><a href="http://www.lyngsat.com/tvchannels/zw/Christ-Revealed-TV.html">Christ Revealed TV</a> <font size=1><br>on <a href="https://www.lyngsat.com/Intelsat-20.html">Intelsat 20</font></a></font></td></tr><tr><td align="center" width=32 height=32><img src="/logo/tv/rr/raapchik-in.png" width=32 height=24></td><td align="left" width=175><font face="Arial" size=2><a href="http://www.lyngsat.com/tvchannels/in/Raapchik.html">Raapchik</a> <font size=1><br>on <a href="https://www.lyngsat.com/G-Sat-15.html">G-Sat 15</font></a></font></td></tr></table>
</td>
</tr>
</table>
</div><div class="mobile"></div><br>
</div>
<table width=720 border cellspacing=0 cellpadding=0>
<tr>
<td class="td-copyright" colspan=10 align="center"><font face="Arial" size=2>Optus D3 &#169; LyngSat, last updated 2024-04-02 - https://www.lyngsat.com/Optus-D3.html</td>
</tr>
<tr>
<td width=70 align="center"><font face="Verdana"><font size=2>Frequency</font><font size=1><br>Beam<br>EIRP (dBW)</font></td>
<td width=60 align="center"><font face="Verdana"><font size=1>System<br>SR<br>FEC</font></td>
<td width=52 align="center"><font face="Verdana"><font size=1>Logo<br>SID</font></td>
<td width=200 align="left"><font face="Verdana" size=2><i>Provider Name</i><br>Channel Name</font></td>
<td width=15><br></td>
<td width=70 align="left"><font face="Verdana" size=1><font color="blue">ONID-TID</font><br>Compression<br>Format</font></td>
<td width=50 align="center"><font face="Verdana" size=1>VPID</font></td>
<td width=50 align="left"><font face="Verdana" size=1><font color="blue">C/N lock</font><br>Audio</font></td>
<td width=60 align="left"><font face="Verdana" size=1>Encryption</font></td>
<td width=80 align="left"><font face="Verdana" size=1>Source<br>Updated</font></td>
</tr>
<tr>
<td style="background:linear-gradient(to right,#ffb6c1 50%,#bbffbb 51%);" align="center"><font face="Verdana"><font size=2><b><a href="https://www.lyngsat.com/muxes/Optus-D3_New-Zealand_12707-H.html" style="color:blue">12707&nbsp;H</a></b><br><font size=1>tp NZ16</font><br></font><font size=1>New Zealand</font></td>
<td style="background:#ffffbb" align="center"><font face="Verdana" size=1>DVB-S<br>22500</font></td>
<td style="background:#ffffbb" align="center" width=52><font size=1 face="Verdana"></font></td>
<td style="background:#ffffbb"><font face="Arial"><font size=2 color="gray"><b>[Sky New Zealand test card]</b></td>
<td style="background:#ffffbb"><br></td>
<td style="background:#ffffbb"><font face="Verdana" size=1>MPEG-2/SD</td>
<td align="center" style="background:#ffffbb"><font face="Verdana" size=1>1001</font></td>
<td style="background:#ffffbb"><font face="Verdana" size=1>1101&nbsp;<br></font></td>
<td style="background:#ffffbb"><font face="Verdana" size=1></font></td>
<td><font face="Verdana" size=1>D Lee 4<br>140806</font></td>
</tr>
<tr>
<td class="td-copyright" colspan=10 align="center"><font face="Arial" size=2>Optus D3 &#169; LyngSat, last updated 2024-04-02 - https://www.lyngsat.com/Optus-D3.html</td>
</tr>
</table>
<br>
<table align="center">
<tr>
<td colspan=5 align="center"><font face="Verdana" size=1><b>Colour legend:</b> (based on your location)</td>
</tr>
<tr>
<td><font face="Verdana" size=1><b>Beam:</b></td>
<td style="background:#bbffbb"><font face="Verdana" size=1>Within footprint center</td>
<td style="background:#ffffbb"><font face="Verdana" size=1>Within footprint</td>
<td style="background:#ffcc99"><font face="Verdana" size=1>Outside of footprint</td>
<td style="background:#ffb6c1"><font face="Verdana" size=1>Below horizon</td>
<td style="background:white"><font face="Verdana" size=1>Beam data unavailable</td>
</tr>
<tr>
<td><font face="Verdana" size=1><b>Frequency:</b></td>
<td style="background:#bbffbb"><font face="Verdana" size=1>Ku band</td>
<td style="background:#ffffbb"><font face="Verdana" size=1>C band</td>
<td style="background:#ffcc99"><font face="Verdana" size=1>S/Ka band</td>
<td style="background:#ffb6c1"><font face="Verdana" size=1>-</td>
<td style="background:white"><font face="Verdana" size=1>-</td>
</tr>
<tr>
<td><font face="Verdana" size=1><b>System:</b></td>
<td style="background:#bbffbb"><font face="Verdana" size=1>DVB-S2</td>
<td style="background:#ffffbb"><font face="Verdana" size=1>DVB-S</td>
<td style="background:#ffcc99"><font face="Verdana" size=1>non-DTH</td>
<td style="background:#ffb6c1"><font face="Verdana" size=1>-</td>
<td style="background:white"><font face="Verdana" size=1>-</td>
</tr>
<tr>
<td><font face="Verdana" size=1><b>Access:</b></td>
<td style="background:#bbffbb"><font face="Verdana" size=1>Clear, HD</td>
<td style="background:#ffffbb"><font face="Verdana" size=1>Clear, SD</td>
<td style="background:#ffcc99"><font face="Verdana" size=1>Encrypted, subscribable</td>
<td style="background:#ffb6c1"><font face="Verdana" size=1>Encrypted, non-subscribable</td>
<td style="background:white"><font face="Verdana" size=1>-</td>
</tr>
<tr>
<td colspan=5 align="center"><font face="Verdana" size=1><i>The subscription info per country may be incomplete.</i></td>
</tr>
</table>

<p>
<table width=720 border=0 cellspacing=0 cellpadding=0>
<tr>
<td><img src="/images/pixel_brun.gif" width=100% height=1 vspace=7></td>
</tr>
<tr align="center">
<td>
<table border=0 cellspacing=0 cellpadding=0>
<tr align="center">
<td align="center">
<div class="desktab">
<div class="desktop"><table width=728 border=0 cellspacing=1 cellpadding=1><tr><td align='center' width=728 height=90><script async src="//pagead2.googlesyndication.com/pagead/js/adsbygoogle.js"></script>
<!-- LyngSat 728x90 #2 -->
<ins class="adsbygoogle"
     style="display:inline-block;width:728px;height:90px"
     data-ad-client="ca-pub-9851824455959986"
     data-ad-slot="8866439388"></ins>
<script>
(adsbygoogle = window.adsbygoogle || []).push({});
</script></td>
</tr></table></div><div class="tablet"><table width=728 border=0 cellspacing=1 cellpadding=1><tr><td align='center' width=728 height=90><style>
.max728x90 {width:728px; height:90px;}
</style>
<script async src="https://pagead2.googlesyndication.com/pagead/js/adsbygoogle.js"></script>
<!-- LyngSat #2 -->
<ins class="adsbygoogle max728x90"
     style="display:block"
     data-ad-client="ca-pub-9851824455959986"
     data-ad-slot="1366621015">
</ins>
<script>
     (adsbygoogle = window.adsbygoogle || []).push({});
</script></td>
</tr></table></div><div class="mobile"></div></div>
<br>
</td>
</tr>
</table>
</td>
</tr>
</table>
<table width=720 border=0 cellspacing=0 cellpadding=0>
<tr>
<td align="center">
<table>
<tr><font face="Arial" size=2>
The content on this website is protected by copyright. All rights reserved.<br>
LyngSat is a registered trademark, owned by LyngSat Network.</font>
</font></tr>
</table>
<br>
<font face="Arial" size=2>You are very welcome to send updates and corrections to <b><a href="mailto:webmaster@lyngsat.com">webmaster@lyngsat.com</a></b><br>
<br>
</td>
</tr>
</table>
<td width="160" align="center" valign="top">
<div class="desktop">
<table border=0 cellspacing=1 cellpadding=1>
<tr><td align='center'><font face='Arial' size=2><a href='http://www.lyngsat.com/advert.html'><font color='blue'><b>Advertisements</b></font></a></font></td></tr><tr align='center'>
<td width=160 height=600><script async src="//pagead2.googlesyndication.com/pagead/js/adsbygoogle.js"></script>
<!-- LyngSat 160x600 #1 -->
<ins class="adsbygoogle"
     style="display:inline-block;width:160px;height:600px"
     data-ad-client="ca-pub-9851824455959986"
     data-ad-slot="1496632095"></ins>
<script>
(adsbygoogle = window.adsbygoogle || []).push({});
</script></td>
</tr>
</table>
</div>
</td>
</tr>
</table>
<!-- big table end -->
</div>
</body>
</html>

'''

## Adjusting the Scraping Methodology

In [ ]:
def is_FEC(sus_txt):
    match = re.search('^\d/\d', sus_txt)
    if match:
        return True
    return False

In [ ]:
# is_Pkg("https://www.lyngsat.com/packages/VAST.html")

In [ ]:
print(is_FEC('2/3'))


    

In [ ]:

def extract_raw_data_alt(column_data,recurse_thru_a = False, find_tags = 'td', extract_provider_links = False, pattern =''):
    raw_data = []
    for row in column_data:
        row_data = row.find_all(find_tags)
        columns = row_data
        individual_row_data = []
        
        
        if(recurse_thru_a == True):
            extracted_links = [data.find('a')['href'] if data.find('a') else data.text.strip() for data in row_data]
            link_to_page = extracted_links[0] if extracted_links else None
            individual_row_data.append(link_to_page)

        individual_row_data +=[data.text.strip() for data in row_data]
        
        # for data in row_data:
        font = columns[1].find_all('font')[0]
        # font = font.replace('<br/>', ',')
        br = font.find_all('br')
        
        system = None
        SR = None
        FEC = None
        
        len_br = len(list(enumerate(br)))
        # for i, thing in enumerate(br):
        for i, thing in enumerate(br):  
            # print(f'{i} -> font: {font}')
            # print(f'{i}: {thing}')    
            if i == 0:
                system = thing.previous_sibling.get_text(strip = True)
            if len_br == 3:
                if i == 1:
                    SR = thing.next_sibling.get_text(strip = True) if thing.next_sibling else None
                elif i == 2:
                    if thing.next_sibling:
                        fec_flag = is_FEC(thing.next_sibling.get_text(strip = True))
                        if fec_flag == True:
                            FEC = thing.next_sibling.get_text(strip = True)
                        else:
                            SR = thing.next_sibling.get_text(strip = True)
            elif len_br == 2:
                if thing.previous_sibling:
                    # print(f'prev: {thing.previous_sibling}')
                    SR = thing.previous_sibling.get_text(strip = True)
                if thing.next_sibling:
                    if(is_FEC(thing.next_sibling.get_text(strip = True))):
                        FEC = thing.next_sibling.get_text(strip = True)
                    else:
                        SR = thing.next_sibling.get_text(strip = True)
                    
        individual_row_data.append(system)
        individual_row_data.append(SR)
        individual_row_data.append(FEC)
                
        # print(f'system: {system}, SR: {SR}, FEC: {FEC}')
                
            # print(f'{i}: {thing.previous_sibling.get_text(strip=True)}')
        
            # if i == len(br):
            #     print('last element')

        
        # print(f'joined_text: {br}')
           
        # print(len(columns))
        if len(columns) == 10:
            font = columns[0].find_all('font')[0]
            # print('font: ', font)
            
            br = font.find_all('br')
            # print('br: ', br)
            freq = None
            beam = None
            eirp = None
            for i, thing in enumerate(br):
                if i == 0:
                    freq = thing.previous_sibling.get_text(strip=True)
                elif i == 1:
                    beam = thing.previous_sibling.get_text(strip=True)
                else:
                    eirp = thing.next_sibling.get_text(strip=True) if thing.next_sibling else None
            # print(f'freq: {freq}, beam: {beam}, eirp: {eirp}')
        individual_row_data.append(freq)
        individual_row_data.append(beam)
        individual_row_data.append(eirp)
        
        if len(columns) == 10:
            fonts = columns[8].find_all('font')
            # print('fonts for network: ', fonts)
            fonts = [bs(str(data).replace('<br/>',',')).text for data in fonts]
            first_font = fonts[0] if fonts else None
            individual_row_data.append(first_font)
        else:
            fonts = columns[6].find_all('font')
            fonts = [bs(str(data).replace('<br/>',',')).text for data in fonts]
            first_font = fonts[0] if fonts else None
            individual_row_data.append(first_font)
            # individual_row_data += fonts.pop()
        # print('fonts: ', fonts)
                        
        # individual_row_data +=[data.text.strip() for data in row_data]
        
        # individual_row_data +=[BeautifulSoup(str(data).replace('<br/>',',')).text for data in row_data]
        
        if extract_provider_links == True:
            extracted_links = [data.find('a')['href'] if data.find('a') else data.text.strip() for data in row_data]
            for element in extracted_links:
                matches = re.findall(pattern, element)
                if(matches):
                    individual_row_data.append(element)

        raw_data.append(individual_row_data)
    return raw_data

In [ ]:
soup = bs(sat_html, 'html')

sat_table = soup.find_all('table',{'border':"", 'cellpadding':"0", 'cellspacing':"0", 'width':"720"} )

In [ ]:
column_data = extract_rows_from_tables(sat_table)

for i, table in enumerate(column_data):
#     print("table#",i)
    if(len(table) > 2):
        _temp = column_data[i]
        _temp = _temp[2:len(_temp)-1]
    #     print(_temp)
        column_data[i] = _temp

# Flattens the list. Now, we have a list of tr tags
column_data = flatten_comprehension(column_data)

# Clean and extract the data values from the tags
channels_extracted = extract_raw_data_alt(column_data)
pkgs_extracted = extract_providers_link(column_data, pattern='.*packages')

# pkgs_extracted = extract_providers_link(column_data, pattern='.*packages')
chan_links_df = pd.DataFrame(data=pkgs_extracted)
chan_links_df.head(1)

In [ ]:
chan_links_df = chan_links_df.iloc[:, [3,21]]
chan_links_df.rename(columns ={3: "Pkg_Name", 21: "Pkg_URL"}, inplace=True)


In [ ]:
chan_links_df

In [ ]:
merged_df.head(3)

In [ ]:
import csv
# import MySQLdb
import mysql.connector


In [ ]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="root",
  database="lyngsat"
)

In [ ]:
mydb

In [ ]:
# print(mydb)
cursor = mydb.cursor()


In [ ]:
# cursor.execute("SHOW DATABASES")

cursor.execute('SELECT * FROM channel')

In [ ]:
result = cursor.fetchall()
result


In [ ]:
channels_df.shape

In [ ]:
# result
channels_df = pd.read_csv('E:/AUC/23-24/Spring/Database/Project/CSV Files/entities/v6_channels.csv')
networks_df = pd.read_csv('E:/AUC/23-24/Spring/Database/Project/CSV Files/entities/v6_networks.csv')

channels_df.shape

In [ ]:
def max_char_count(df_column):
    lengths = df_column.str.len()
    # Find the index of the longest string
    index_of_max = lengths.idxmax()
    print(index_of_max, ', ',df_column.iloc[index_of_max])
    return df_column.str.len().max()

def display_max_length(df):
    for i, col in enumerate(df.keys()):
        char_count = max_char_count(df[col].astype(str))
        print(f'COLUMN: {col},  MAX_LENGTH: {char_count}')


def match_first_part(string):
    match = re.search(r'^[^(]+', string)
    if match:
        return match.group()
    else:
        return string
        

In [ ]:
string = "SWR Fernsehen Rheinland-Pfalz"
# channels_df['Provider/Channel'].apply(match_first_part)
channels_df.drop_duplicates(subset=['Provider/Channel'], inplace=True)

channels_df.loc[:, 'Provider/Channel'] = channels_df['Provider/Channel'].apply(match_first_part)

# print(match_first_part(string))

In [ ]:
# channels_df['Channel_Country'].isna().value_counts()
channels_df['Channel_Country'].fillna('NA', inplace=True)
records_to_insert = channels_df.to_records(index=False).tolist()


# channels_df[channels_df['Provider/Channel'].isna()]

# display_max_length(networks_df)

# display_max_length(channels_df)
# networks_df.drop_duplicates(subset=['', 'B'], inplace=True)
# Keep only the first occurrence of each duplicate based on columns 'A' and 'B'
# networks_df_dupl = networks_df[networks_df.duplicated(subset=['Provider'], keep='first')]


In [ ]:
# networks_df['Provider'].replace('Ã', 'A')
# networks_df['Provider'].replace('Ü', 'U')

replacement_dict = {'Ã': 'A', 'Ü': 'U', 'ó':'o','é':'e'}

networks_df = pd.read_csv('C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/v6_networks_cleaned.csv')

# Apply the replacements to the 'Provider' column
networks_df['Provider'] = networks_df['Provider'].replace(replacement_dict, regex=True)

# networks_df.to_csv('C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/v6_networks_cleaned_1.csv',index = False, encoding='UTF-8')
networks_df


In [ ]:
sql = "INSERT INTO channel (Name, Country) VALUES (%s, %s)"
# val = ("John", "Highway 21")
# cursor.execute(sql, val)

## Cleaning Channel Instances

In [ ]:
# channels_df.drop_duplicates(subset=['Provider/Channel'], inplace=True)

# channels_df.to_csv('E:/AUC/23-24/Spring/Database/Project/CSV Files/entities/v6_channels_cleaned.csv', encoding='UTF-8', index=False)
channel_instances_df = pd.read_csv('C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/v6_chan_instances_cleaned.csv')

In [ ]:
channel_instances_df.keys()
# channel_instances_df[channel_instances_df['Pkg_Name'].notna()]

In [ ]:
instances_df = channel_instances_df.drop(columns=['VPID','Unnamed: 0', 'encryption_alt', 'Channel','Languages','Provider_URL', 'Channel', 'Channel_URL',
       'is_Provider','Channel_Country','Provider_Country','Freq/beam', 'SR/FEC', 'undef', 'Audio', 'Encryption', 'Src_Updated','Pkg_Name', 'Pkg_URL', ])

instances_df.loc[:, 'Provider/Channel'] = instances_df['Provider/Channel'].apply(match_first_part)

# instances_df.head(0)
display_max_length(instances_df)

In [ ]:


# instances_df = instances_df[['Provider/Channel','Sattelite_Name', 'freq_alt', 'beam_alt', 'sr_alt', 'fec_alt','Compression','eirp_alt']]
instances_df.head(2)

In [ ]:
display_max_length(instances_df)



In [ ]:
instances_df.head(1)

instances_df['Sattelite_Name'] = instances_df['Sattelite_Name'].apply(get_normal_sat_name)
# instances_df.drop_duplicates(subset=['Provider/Channel', 'Sattelite_Name', 'freq_alt'],inplace=True)

In [ ]:
instances_df['Sattelite_Name']

In [ ]:
instances_df.shape
# channels_df[0:20]
# instances_df.drop_duplicates(subset=['Provider/Channel', 'Sattelite_Name', 'freq_alt'])

instances_df.to_csv('C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/v6_chan_instances_cleaned_1.csv', index = False)

In [ ]:
# instances_df.shape
# records_to_insert = channels_df.to_records(index=False).tolist()
instances_df.loc[:, 'Provider/Channel'] = instances_df['Provider/Channel'].apply(match_first_part)

# cursor.executemany(sql, records_to_insert)
# mydb.commit()
# print(cursor.rowcount, "Record inserted successfully into channel table")

In [ ]:
instance_lng_df = pd.read_csv('C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/v6_channels_instance_language.csv')

instance_lng_df.head(0)
instance_lng_df.shape

In [ ]:
instance_lng_df.loc[:, 'Provider/Channel'] = instance_lng_df['Provider/Channel'].apply(match_first_part)


In [ ]:
instance_lng_df.drop_duplicates(subset=['Provider/Channel', 'Sattelite_Name', 'freq_alt', 'Languages'],inplace=True)

instance_lng_df.shape

In [ ]:
instance_lng_df.to_csv('C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/v6_chan_instances_lng_cleaned.csv', index = False)

In [ ]:
instance_enc_df = pd.read_csv('C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/v6_channels_instance_encryption.csv')
instance_enc_df.shape


In [ ]:
instance_enc_df.head(0)

In [ ]:
instance_enc_df.loc[:, 'Provider/Channel'] = instance_enc_df['Provider/Channel'].apply(match_first_part)

In [ ]:
instance_enc_df.drop_duplicates(subset=['Provider/Channel', 'Sattelite_Name', 'freq_alt', 'encryption_alt'],inplace=True)

instance_enc_df.shape

In [ ]:
instance_enc_df.to_csv('C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/v6_chan_instances_enc_cleaned.csv', index = False)

In [ ]:
instance_lng_df = pd.read_csv('C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/v6_chan_instances_lng_cleaned.csv')


instance_lng_df['Sattelite_Name'] = instance_lng_df['Sattelite_Name'].apply(get_normal_sat_name)

instance_lng_df
instance_lng_df.to_csv('C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/v6_chan_instances_lng_cleaned_1.csv', index = False)

In [ ]:
instance_enc_df = pd.read_csv('C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/v6_chan_instances_enc_cleaned.csv')


instance_enc_df['Sattelite_Name'] = instance_enc_df['Sattelite_Name'].apply(get_normal_sat_name)

instance_enc_df
# instance_enc_df.to_csv('C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/v6_chan_instances_enc_cleaned_1.csv', index = False)

In [ ]:
instance_enc_df.dropna(subset=['encryption_alt'], inplace=True)

instance_enc_df.to_csv('C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/v6_chan_instances_enc_cleaned_1.csv', index = False)
# instance_enc_df.keys()